# Section 1: Preprocessing
## Behavior Analysis
### Generate trial regressors

In [ ]:
import os
import numpy as np
from pandas import concat, read_csv
from scipy.stats import gamma
def normalize(arr): return (arr - arr.min()) / (arr.max() - arr.min())

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior'
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
threshold = 0.005

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load / Concatenate / Prepare Data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
df = []
for subject in subjects:
    
    ## Load CSV.
    csv = read_csv(os.path.join(root_dir,'%s_msit_data.txt' %subject))
    
    ## Limit columns.
    csv = csv[['SubjID','trial','iaps','DBS','interference','valence','arousal','responseTime','responseCorrect']]
    
    ## Rename columns.
    csv.columns = ['Subject', 'Trial', 'IAPS', 'DBS', 'Interference', 'Valence_Obj', 'Arousal_Obj', 'RT', 'Accuracy']

    ## Load IAPS ratings.
    iaps = read_csv(os.path.join(root_dir,'%s_IAPS_SAM.csv' %subject))
    iaps = iaps[['IAPS_Number','Valence','Arousal']]
    iaps.columns = ['IAPS','Valence_Subj','Arousal_Subj']

    ## Merge. Append.
    csv = csv.merge(iaps, on='IAPS')
    cols = ['Subject', 'Trial', 'IAPS', 'DBS', 'Interference', 'Valence_Obj', 'Arousal_Obj', 
            'Valence_Subj', 'Arousal_Subj', 'RT', 'Accuracy']
    csv = csv[cols]
    df.append(csv)

## Merge data. Sort.
df = concat(df)
df['DBS'] = np.where(df['DBS']=='DBSoff',0,1)
df = df.sort_values(['Subject','DBS','Trial']).reset_index(drop=True)

## Normalize regressors.
df['nsArousal'] = normalize(df.Arousal_Subj)
df['nsValence'] = normalize(df.Valence_Subj)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Determine Trials for Inclusion/Exclusion.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Set missing RTs to NaNs.
df['RT'] = np.where(df.Accuracy==-1, np.nan, df.RT)
df['Accuracy'] = np.where(df.Accuracy==-1, np.nan, df.Accuracy)
df['Missing'] = df.Accuracy.isnull().astype(int)

## Add Error column.
df['Error'] = 1 - df.Accuracy

## Add Post-Error Column.
df['PostError'] = 0
for subject in df.Subject.unique():
    error = df.loc[df.Subject==subject,'Error']
    posterror = np.insert(np.roll(error,1)[1:], 0, 0)
    df.loc[df.Subject==subject,'PostError'] = posterror

## Iteratively detect outliers across subjects by fitting a Gamma distribution.
df['GammaCDF'], df['Outlier'] = 0, 0
for subject in df.Subject.unique():
    
    ## Fit Gamma to reaction time distribution.
    shape, loc, scale = gamma.fit(df.loc[(df.Subject==subject)&(~df.RT.isnull()),'RT'], floc=0)
    
    ## Find outliers given likelihood threshold.
    cdf = gamma.cdf(df.loc[(df.Subject==subject)&(~df.RT.isnull()),'RT'], shape, loc=loc, scale=scale)
    outliers = (cdf < threshold) | (cdf > 1 - threshold)
    
    ## Append information.
    df.loc[(df.Subject==subject)&(~df.RT.isnull()), 'GammaCDF'] += cdf
    df.loc[(df.Subject==subject)&(~df.RT.isnull()), 'Outlier'] += outliers.astype(int)
    
## Generate exclude.
df['Exclude'] = np.where( df[['Missing','Error','PostError','Outlier']].sum(axis=1), 1, 0)
print '%s trials (%0.2f%%) excluded.' %(df.Exclude.sum(), df.Exclude.mean())

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
df.to_csv('%s/afMSIT_group_data.csv' %root_dir, index=False)

## Parcellation
### Make EMOTE Labels

In [ ]:
import os, shutil
import numpy as np
import pylab as plt
from mne import read_label, read_source_spaces, read_surface, set_log_level
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fs_dir = '/space/sophia/2/users/EMOTE-DBS/freesurfs'
subject = 'BRTU'

parc = 'laus250'
label_dir = os.path.join(fs_dir,subject,'label',parc)
out_dir = os.path.join(fs_dir,subject,'label','april2016')

if os.path.isdir(out_dir): shutil.rmtree(out_dir)
os.makedirs(out_dir)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Build Left Hemisphere Labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

hemi = 'lh'
rr, _ = read_surface(os.path.join(fs_dir, subject, 'surf', '%s.inflated' %hemi))
src = read_source_spaces(os.path.join(fs_dir, subject, 'bem', '%s-oct-6-src.fif' %subject))[0]

lhdict = {'dlpfc_1-lh':['caudalmiddlefrontal_1', 'caudalmiddlefrontal_5', 'caudalmiddlefrontal_6'],
          'dlpfc_2-lh':['caudalmiddlefrontal_2', 'caudalmiddlefrontal_3', 'caudalmiddlefrontal_4'],
          'dlpfc_3-lh':['rostralmiddlefrontal_2', 'rostralmiddlefrontal_3'],
          'dlpfc_4-lh':['rostralmiddlefrontal_1', 'rostralmiddlefrontal_5'],
          'dlpfc_5-lh':['parstriangularis_2', 'parsopercularis_2'],
          'dlpfc_6-lh':['parsopercularis_3', 'parsopercularis_4'],
          'racc-lh':['rostralanteriorcingulate_1','rostralanteriorcingulate_2'],
          'dacc-lh':['caudalanteriorcingulate_1','caudalanteriorcingulate_2',],
          'pcc-lh':['posteriorcingulate_2','posteriorcingulate_3']}

for k,V in lhdict.iteritems():
    
    label = np.sum([read_label(os.path.join(label_dir,'%s-%s.label' %(v,hemi)), subject=subject) 
                    for v in V])
    n_vert = np.intersect1d(src['vertno'], label.vertices).shape[0]
    print '%s\t%s' %(n_vert,k)
    label.save(os.path.join(out_dir, '%s.label' %k))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Build Right Hemisphere Labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

hemi = 'rh'
rr, _ = read_surface(os.path.join(fs_dir, subject, 'surf', '%s.inflated' %hemi))
src = read_source_spaces(os.path.join(fs_dir, subject, 'bem', '%s-oct-6-src.fif' %subject))[1]

rhdict = {'dlpfc_1-rh':['caudalmiddlefrontal_1', 'caudalmiddlefrontal_2', 'caudalmiddlefrontal_5'],
          'dlpfc_2-rh':['caudalmiddlefrontal_3', 'caudalmiddlefrontal_4'],
          'dlpfc_3-rh':['rostralmiddlefrontal_2', 'rostralmiddlefrontal_3'],
          'dlpfc_4-rh':['rostralmiddlefrontal_1', 'rostralmiddlefrontal_5'],
          'dlpfc_5-rh':['parstriangularis_2', 'parsopercularis_1'],
          'dlpfc_6-rh':['parsopercularis_3', 'parsopercularis_4'],
          'racc-rh':['rostralanteriorcingulate_1','rostralanteriorcingulate_2'],
          'dacc-rh':['caudalanteriorcingulate_1','caudalanteriorcingulate_2','caudalanteriorcingulate_3'],
          'pcc-rh':['posteriorcingulate_2','posteriorcingulate_3']}

for k,V in rhdict.iteritems():
    label = np.sum([read_label(os.path.join(label_dir,'%s-%s.label' %(v,hemi)), subject=subject) 
                    for v in V])
    n_vert = np.intersect1d(src['vertno'], label.vertices).shape[0]
    print '%s\t%s' %(n_vert,k)
    label.save(os.path.join(out_dir, '%s.label' %k))

### Custom dmPFC Labels
Similarly made from Lausanne atlas. 

In [ ]:
import os
from mne import read_label, split_label

laus_dir = '/space/lilli/1/users/DARPA-Recons/fscopy/label/laus125/'
out_dir  = '/space/sophia/2/users/EMOTE-DBS/freesurfs/fscopy/label/april2016'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Build Left Hemisphere Labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Initialize with superiorfrontal_4-lh
lh_label = read_label(os.path.join(laus_dir, 'superiorfrontal_4-lh.label'), subject='fsaverage')

## Add split labels from superiorfrontal_7 
labels = read_label(os.path.join(laus_dir, 'superiorfrontal_7-lh.label'), subject='fsaverage')
labels = split_label(labels, parts=7)
for label in labels[2:5]: lh_label += label
    
## Add split labels from superiorfrontal_5
labels = read_label(os.path.join(laus_dir, 'superiorfrontal_5-lh.label'), subject='fsaverage')
labels = split_label(labels, parts=4)
lh_label += labels[0]

## Rename save.
lh_label.name = 'dmpfc-lh'
lh_label.save( os.path.join(out_dir,'dmpfc-lh') )

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Build Right Hemisphere Labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Initialize with superiorfrontal_3 (first split)
labels = read_label(os.path.join(laus_dir, 'superiorfrontal_3-rh.label'), subject='fsaverage')
labels = split_label(labels, parts=4)
rh_label = labels[0]

## Add split labels from superiorfrontal_6
labels = read_label(os.path.join(laus_dir, 'superiorfrontal_6-rh.label'), subject='fsaverage')
labels = split_label(labels, parts=4)
for label in labels[:2]: rh_label += label
    
## Add split labels from superiorfrontal_5
labels = read_label(os.path.join(laus_dir, 'superiorfrontal_5-rh.label'), subject='fsaverage')
labels = split_label(labels, parts=4)
for label in labels[:2]: rh_label += label
    
## Rename save.
rh_label.name = 'dmpfc-rh'
rh_label.save( os.path.join(out_dir,'dmpfc-rh') )

### Morph EMOTE labels
Morphing was done in the command line using the following code.

### Check Label Sizes

In [ ]:
import os
import numpy as np
from mne import read_source_spaces, read_label, set_log_level
set_log_level(verbose=False)
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
src = read_source_spaces(os.path.join(fs_dir,'fscopy','bem','fscopy-oct-6p-src.fif'))
rois = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']
labels = [read_label(os.path.join(fs_dir,'fscopy','label','april2016','%s.label' %roi), subject='fsaverage')
          for roi in rois]

for label in labels:
    if label.hemi == 'lh': print np.intersect1d(src[0]['vertno'], label.vertices).shape[0], label.name
    else:  print np.intersect1d(src[1]['vertno'], label.vertices).shape[0], label.name

### Plot EMOTE Labels (Groups)

In [ ]:
import os
from surfer import Brain
%matplotlib qt4

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Make brain.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subject_id = "fscopy"
hemi = "rh"
surf = "inflated"
brain = Brain(subject_id, hemi, surf)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Load files.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
subj_dir = os.environ["SUBJECTS_DIR"]
rois = ['dacc-lh', 'dacc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']
labels = [os.path.join(fs_dir,'fscopy','label','april2016','%s.label' %roi) for roi in rois]
colors =['#810f7c', '#810f7c', 
        '#004529', '#004529', '#006837', '#006837', '#238443', '#238443', '#41ab5d', '#41ab5d', 
         '#78c679', '#78c679', '#addd8e', '#addd8e', '#d9f0a3', '#d9f0a3', '#f7fcb9', '#f7fcb9',
        '#8c6bb1', '#8c6bb1', '#9ebcda', '#9ebcda']

for n, label in enumerate(labels): 
    if hemi in rois[n]:
        brain.add_label(label, color=colors[n], alpha=1)

brain.show_view({'azimuth': 135, 'elevation': 79}, roll=107)

### Plot EMOTE Labels (Individuals)

In [ ]:
import os
from surfer import Brain
%matplotlib qt4

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Make brain.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subject_id = "fscopy"
surf = "inflated"

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Load files.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
subj_dir = os.environ["SUBJECTS_DIR"]
rois = ['dacc-lh', 'dacc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']
labels = [os.path.join(fs_dir,'fscopy','label','april2016','%s.label' %roi) for roi in rois]
colors =['#810f7c', '#810f7c', 
        '#004529', '#004529', '#006837', '#006837', '#238443', '#238443', '#41ab5d', '#41ab5d', 
         '#78c679', '#78c679', '#addd8e', '#addd8e', '#d9f0a3', '#d9f0a3', '#f7fcb9', '#f7fcb9',
        '#8c6bb1', '#8c6bb1', '#9ebcda', '#9ebcda']

for roi, label, color in zip(rois,labels,colors):
    
    hemi = roi.split('-')[-1]
    brain = Brain(subject_id, hemi, surf)

    brain.remove_labels()
    brain.add_label(label, color=color, alpha=1)
    
    if 'pfc' in roi: view = 'lateral'
    else: view = 'medial'
    
    if hemi == 'lh' and view == 'lateral': brain.show_view({'azimuth': 160, 'elevation': 80}, roll=90, distance=450)
    elif hemi == 'rh' and view == 'lateral': brain.show_view({'azimuth': 20, 'elevation': 80}, roll=-85, distance=450)
    elif hemi == 'lh' and view == 'medial': brain.show_view({'azimuth': 10, 'elevation': 100}, roll=-85, distance=450)
    elif hemi == 'rh' and view == 'medial': brain.show_view({'azimuth': 170, 'elevation': 100}, roll=90, distance=450)
    brain.save_image('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/source/anatomy/%s.png' %roi)

### Plot Sensor Layout

In [ ]:
import os
import numpy as np
import pylab as plt
from mne.io import Raw

## Define parameters.
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
subject = 'S2'
raw_file = os.path.join(root_dir,'raw','%s_msit_raw.fif' %subject)

## Open raw. Get channel info.
raw = Raw(raw_file, preload=False, verbose=False)
chs = raw.info['chs']

## Get channel names and locations.
ch_info = dict()
for d in chs:
    if d['loc'].sum():
        ch_info[d['ch_name']] = d['loc'][:3] * 1e3
        
## Iteratively plot.
fig, ax = plt.subplots(1,1,figsize=(8,12))
for k,v in ch_info.iteritems():
    ax.scatter(v[0],v[1],s=125,color='k')
    ax.text(v[0]+2,v[1]+2,k,fontsize=16)
ax.tick_params(axis='both',which='both',bottom='off',left='off',
               labelbottom='off',labelleft='off')
plt.suptitle('Representative Eximia Cap', fontsize=30)
plt.tight_layout()
# plt.show()
plt.savefig(os.path.join(root_dir,'plots','sensor','eximia_cap.png'))
plt.close()

## Preprocesing 1: Raw Data
### Fixing MEWA: Digitization
Something got way messed up. Here we make MNE knows what is EEG and what is extra points.

NOTE: Copied over one of the original files for MEWA and renamed it MEWA_msit_unmasked_raw.fif

In [ ]:
import os
import numpy as np
from mne.io import Raw
from pandas import read_table

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
raw_file = 'MEWA_msit_unmasked_raw.fif'
out_file = 'MEWA_msit_raw.fif'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare digitizations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load data. Get digitization from raw.
raw = Raw(os.path.join(root_dir,'raw',raw_file),preload=False,verbose=False)
digitization = raw.info['dig']

## The last 101 points are extra. Set them to kind=4.
for d in digitization[-101:]: d['kind'] = 4
    
## Get coordinates for EEG points (excluding ref/EOG).
rr = np.array([d['r'] for d in dig if d['kind']==3])[:-2]

## Get channels
chs = raw.info['chs']

## Update location information. This was a huge pain in the ass to figure out.
## We ignore the first four channels (Triggers, EOG) and the last channel (STI014).
for ch, r in zip(chs[4:-1], rr): ch['loc'][:3] = r 

## Update digitization/chs.
raw.info['dig'] = digitization
raw.info['chs'] = chs
raw.save(os.path.join(root_dir,'raw',out_file), overwrite=True)

### Fixing MEWA: Masking channel jumps
Time windows were manually inspected. This step isn't strictly necessary but seemed to help with EOG projections.

NOTE: Copied over one of the original files for MEWA and renamed it MEWA_msit_unmasked_raw.fif

In [ ]:
import os
import numpy as np
import pylab as plt
from mne.io import Raw, RawArray

## Specify parameters.
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
raw_file = 'MEWA_msit_unmasked_raw.fif'

## Load data.
raw = Raw(os.path.join(root_dir,'raw',raw_file),preload=True,verbose=False)

## Get data in matrix form.
data = raw._data

## Get list of usuable channels
ch_info = [(n,ch) for n,ch in enumerate(raw.ch_names)]
good_ch = [(n,ch) for n,ch in ch_info if ch not in raw.info['bads']]
good_ch = np.array(good_ch)[4:-1]

## Make mask.
mask = np.zeros(data.shape[1])
times = [(384,394), (663,669)]
for t1, t2 in times:
    mask[(raw.times >= t1) & (raw.times <= t2)] += 1
mask = mask.astype(bool)

## Apply mask.
for ch in good_ch[:,0].astype(int):
    data[ch,mask] = 0

## Make new array. Save.
raw = RawArray(data, raw.info, first_samp=raw.first_samp)
raw.add_eeg_average_proj()
raw.save(os.path.join(root_dir,'raw','MEWA_msit_raw.fif'), overwrite=True, verbose=False)

### Projections: EOG

In [ ]:
import os
from mne import write_proj
from mne.preprocessing import compute_proj_eog
from mne.io import Raw

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## File params.
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
subjects = ['MEWA']

# NOTE: Not all subjects work with EOG channel = EOG. 
# Some require other frontal channels due to concatenation.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
for subj in subjects:    
    print 'Making EOG file for %s.' %subj
    
    ## Load files.
    raw_file  = os.path.join( root_dir, 'raw', '%s_msit_raw.fif' %subj )
    raw = Raw(raw_file, preload=True, verbose=False, add_eeg_ref=False)
    raw.del_proj(0)
    ## Make EOG proj. Save.
    proj, _ = compute_proj_eog(raw, n_eeg = 4, average=True, filter_length='20s',
                               reject=dict(eeg=5e-4), flat=dict(eeg=5e-8),  ch_name='F2', n_jobs=3)
    write_proj(os.path.join( root_dir, 'raw', '%s_msit_eog-proj.fif' %subj ), proj)

### Projections: ECG

In [ ]:
import os
from mne import read_proj, write_proj
from mne.preprocessing import compute_proj_ecg
from mne.io import Raw

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## File params.
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
subjects = ['CHDR']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
for subj in subjects:    
    print 'Making ECG file for %s.' %subj
    
    ## Load files.
    raw_file = os.path.join( root_dir, 'raw', '%s_msit_raw.fif' %subj )
    eog_file = os.path.join( root_dir, 'raw', '%s_msit-proj.fif' %subj )
    raw = Raw(raw_file, preload=True, verbose=False)
    eog_proj = read_proj(eog_file)
    raw.add_proj(eog_proj, remove_existing=True)
    raw.apply_proj()
    
    ## Make ECG proj. Save.
    ecg_proj, _ = compute_proj_ecg(raw, n_eeg = 4, h_freq = 35., average=True, filter_length='20s',
                                reject=dict(eeg=5e-4), flat=dict(eeg=5e-8), ch_name='P9', n_jobs=3)
    proj = eog_proj + [ecg for ecg in ecg_proj if ecg['desc'] not in [eog['desc'] for eog in eog_proj]]
    write_proj(os.path.join( root_dir, 'raw', '%s_msit-proj.fif' %subj ), proj)

## Preprocessing 2: Epoching
### Make Forward Solutions

In [ ]:
import os
from mne import read_trans, read_bem_solution, read_source_spaces
from mne import make_forward_solution, write_forward_solution
from mne.io import Raw

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'

## Main loop.
root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT_april2016'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'

for subject in subjects:
    
    print 'Making forward solution for %s.' %subject
    
    ## Load files.
    raw = Raw(os.path.join(root_dir, 'raw', '%s_msit_raw.fif' %subject), preload=False, verbose=False)
    trans = read_trans(os.path.join(fs_dir,subject,'mri','T1-neuromag','sets','COR-%s.fif' %subject))
    src = read_source_spaces(os.path.join(fs_dir,subject,'bem','%s-oct-6p-src.fif' %subject), verbose=False)
    bem = read_bem_solution(os.path.join(fs_dir,subject,'bem','%s-5120-5120-5120-bem-sol.fif' %subject), verbose=False)
    
    ## Compute and save forward solution.
    make_forward_solution(raw.info, trans, src, bem, fname=os.path.join(root_dir,'fwd','%s_msit-fwd.fif' %subject),
                          meg=False, eeg=True, mindist=1.0, overwrite=True, n_jobs=3, verbose=False)

print 'Done.'

### Make Epochs

In [ ]:
import os
import numpy as np
from mne import compute_covariance, Epochs, EpochsArray, find_events, read_proj, pick_types, set_log_level
from mne.io import Raw
from pandas import read_csv
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'

## Filtering parameters.
l_freq = 0.5
h_freq = 50
l_trans_bandwidth = l_freq / 2.
h_trans_bandwidth = 1.0
filter_length = '20s'
n_jobs = 3

## Epoching parameters.
event_id = dict( FN=1, FI=2, NN=3, NI=4 )      # Alik's convention, isn't he smart!?
tmin = -1.5                                    # Leave some breathing room.
tmax =  3.4                                    # Trial is 1900ms, leave 1500ms of room.
resp_buffer = 1.5                              # 1500ms on either side of response.
baseline = (-0.5,-0.1)
reject_tmin = -0.5
reject_tmax = 1.9
reject   = dict(eeg=150e-6)
flat     = dict(eeg=5e-7)
detrend = None
decim = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load behavior.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT'
data_file = os.path.join( root_dir, 'behavior', 'afMSIT_group_data.csv' )
df = read_csv(data_file)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load behavior.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for subj in subjects:
    
    print 'Loading data for %s.' %subj

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    # Define paths.
    raw_file  = os.path.join( root_dir, 'raw', '%s_%s_raw.fif' %(subj,task) )
    proj_file = os.path.join( root_dir, 'raw', '%s_%s-proj.fif' %(subj,task) )
    
    # Load data.
    raw = Raw(raw_file,preload=True,verbose=False)
    proj = read_proj(proj_file)
    
    ## Add projections.
    proj = [p for p in proj if 'ref' not in p['desc']]
    raw.add_proj(proj, remove_existing=True)
    raw.add_eeg_average_proj()
    raw.apply_proj()
    print raw.info['projs']
    
    ## Reduce dataframe to subject.
    data = df[df.Subject==subj]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make events.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Identifying events for %s.' %subj,  
    events = find_events(raw, stim_channel='Trig1', output='onset', min_duration=0.25, verbose=False)

    # Error catching.
    if data.shape[0] != events.shape[0]: raise ValueError('Mismatching number of stimulus onsets!')
    print '%s events found.' %events.shape[0]
    
    # Update event identifiers.
    n = 1
    for dbs in [0,1]:
        for cond in [0,1]:
            ix, = np.where((data.DBS==dbs)&(data.Interference==cond))
            events[ix,-1] = n
            n+=1

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Filter
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Applying bandpass filter to raw [%s, %s].' %(l_freq, h_freq)
    
    Fs = raw.info['sfreq']
    raw.filter(l_freq = l_freq, h_freq = h_freq, filter_length=filter_length, n_jobs=n_jobs,
               l_trans_bandwidth=l_trans_bandwidth, h_trans_bandwidth=h_trans_bandwidth)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make stimulus-locked epochs.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    # Build initial epochs object.
    picks = pick_types(raw.info, meg=False, eeg=True, exclude='bads')
    epochs = Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=baseline, picks=picks,
                    reject=reject, flat=flat, reject_tmin=reject_tmin, reject_tmax=reject_tmax, 
                    proj=True, detrend=detrend, decim=decim)
    
    # First round of rejections.
    epochs.drop_bad()                                                       # Remove bad epochs.
    copy = data.ix[[True if not log else False for log in epochs.drop_log]] # Update CSV based on rejections.
    
    '''NOTE: Making a new dataframe copy is just a shortcut for easy indexing between the Pandas 
       DataFrame and the Epochs object. This is due to the three rounds of rejections being 
       applied to the data (e.g. amplitude, behavior exclusion, equalization).'''
    
    # Drop epochs based on behavior.
    epochs.drop(copy.Exclude.astype(bool))
    
    data = data.ix[[True if not log else False for log in epochs.drop_log]]
    print '%s trials remain after rejections.' %(len(epochs))
    print epochs

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make Response-locked epochs.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Making response-locked epochs.'

    # Build response-locked events.
    response_indices = raw.time_as_index(0.4 + data.RT)             # Compensating for MSIT-lock.
    response_events  = epochs.events.copy()
    response_events[:,0] = response_events[:,0] + response_indices

    # Get data.
    arr = epochs.get_data()
    times = epochs.times

    # Calculate lengths of response-locked epochs.
    response_times = data.RT + 0.4                                  # Compensating for MSIT-lock.
    response_windows = np.array([response_times-resp_buffer, response_times+resp_buffer]).T

    # Iteratively build epochs array.
    trials = []
    for n in xrange(len(epochs)):
        mask = (times >= response_windows[n,0]) & (times <= response_windows[n,1])
        trials.append( arr[n,:,mask] )
    trials = np.array(trials).swapaxes(1,2)

    # Finally, make epochs objects.
    resp_epochs = EpochsArray(trials, epochs.info, response_events, tmin=-resp_buffer, event_id=event_id,)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Saving epoch files.'
    epochs.save(os.path.join(root_dir,'ave','%s_%s_%s_stim-epo.fif' %(subj,task,h_freq)))
    resp_epochs.save(os.path.join(root_dir,'ave','%s_%s_%s_resp-epo.fif' %(subj,task,h_freq)))
    data.to_csv(os.path.join(root_dir,'ave','%s_%s_%s-epo.csv' %(subj,task,h_freq)), index=False)
    
    print '\n#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#\n'
    
print 'Done.'

### Make Covariance Matrices / Inverse Solutions / Morph Maps

In [ ]:
import os
from mne import EpochsArray, read_epochs, read_forward_solution, set_log_level
from mne import compute_covariance, write_cov
from mne import compute_morph_matrix, read_source_spaces
from mne.filter import low_pass_filter
from mne.minimum_norm import make_inverse_operator, write_inverse_operator
from scipy.io import savemat
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'

## Analysis parameters.
fmax = 50

## Source localization parameters.
loose = 0.2
depth = 0.8

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Iteratively load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
src = read_source_spaces(os.path.join(fs_dir,'fscopy','bem','fscopy-oct-6p-src.fif'))

for subject in subjects:
    
    print 'Processing %s' %subject
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load files.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Load in files.
    epo_file = os.path.join(root_dir,'ave','%s_msit_%s_stim-epo.fif' %(subject,fmax))
    epochs = read_epochs(epo_file, verbose=False)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Secondary objects.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    fwd = read_forward_solution(os.path.join(root_dir, 'fwd', '%s_%s-fwd.fif' %(subject,task)), 
                                surf_ori=True, verbose=False)
    
    ## Compute/save noise covariance matrix & inverse operator.
    noise_cov = compute_covariance(epochs, tmin=-0.5, tmax=0.0, method='shrunk', n_jobs=1)
    write_cov(os.path.join(root_dir,'cov','%s_%s_%s-cov.fif' %(subject,task,h_freq)), noise_cov)
    inv = make_inverse_operator(epochs.info, fwd, noise_cov, loose=loose, depth=depth, verbose=False)
    write_inverse_operator(os.path.join(root_dir,'cov','%s_%s_%s-inv.fif' %(subject,task,fmax)), inv)

    ## Pre-compute morph matrix.
    vertices_from = [inv['src'][n]['vertno'] for n in xrange(2)]
    vertices_to = [src[n]['vertno'] for n in xrange(2)]
    morph_mat = compute_morph_matrix(subject, 'fsaverage', vertices_from=vertices_from, 
                                     vertices_to=vertices_to,subjects_dir=fs_dir, smooth=25)
    savemat(os.path.join(root_dir, 'morph_maps', '%s-fsaverage_morph.mat' %subject),
            mdict=dict(morph_mat=morph_mat))
    
print 'Done.'

# Section 2: Primary Analyses
## Sensor Space Analysis
### Prepare time domain epochs.

In [ ]:
import os
import numpy as np
from mne import read_epochs
from mne.filter import low_pass_filter
from pandas import read_csv, concat

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
event_id = ['FN', 'FI', 'NN', 'NI']
analysis = 'stim'
h_freq = 50

## Channel definitions. 
pick_channels = ['FCZ','P10']

## Processing parameters.
fmax = 15
decim = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Iteratively load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT'

for n, subject in enumerate(subjects):
    
    ## Load behavioral data.
    csv = read_csv(os.path.join(root_dir,'ave','%s_%s_%s-epo.csv' %(subject,task,h_freq)))

    ## Load in epochs.
    epo_file = os.path.join(root_dir,'ave','%s_%s_%s_%s-epo.fif' %(subject,task,h_freq,analysis))
    epochs = read_epochs(epo_file, verbose=False)
    
    ## Restrict to times/channels of interest.
    if analysis == 'stim': epochs.crop(-0.5,2.0)
    elif analysis == 'resp': epochs.crop(-1.0,1.0)
    epochs.pick_channels(pick_channels)

    ## Extract epochs.
    Fs = epochs.info['sfreq']
    times = epochs.times
    epochs = epochs.get_data().swapaxes(0,1)
    
    ## Merge.
    if not n: 
        trials = epochs
        df = csv
    else:
        trials = np.concatenate([trials,epochs], axis=1)
        df = concat([df,csv])
        
## Apply lowpass filter. 
trials = low_pass_filter(trials, Fs, fmax, filter_length='20s', n_jobs=3, verbose=False)
df = df.reset_index(drop=True)

## Downsample.
trials = trials[:,:,::decim]
times = times[::decim]

## Convert to uV.
trials *= 1e6

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Make directories (if not already made). Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Make directory.
out_dir = os.path.join(root_dir, 'sensor')
if not os.path.isdir(out_dir): os.makedirs(out_dir)
    
## Save data.
for arr, ch in zip(trials,pick_channels): np.savez_compressed(os.path.join(out_dir, 'afMSIT_sensor_%s_%s_%s' %(analysis,ch,fmax)),
                                                              data=arr, times=times, fmax=fmax)
df.to_csv(os.path.join(out_dir, 'afMSIT_sensor_info.csv'), index=False)

print 'Done.'

### Prepare power amplitudes for single-trial epochs.

In [ ]:
import os
import numpy as np
from mne import read_epochs
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
event_id = ['FN', 'FI', 'NN', 'NI']
h_freq = 50

## Channel definitions. 
pick_channels = ['FCZ','P10']

## Time-frequency parameters.
fdict = dict(theta = (4,8), alpha = (8,15), beta = (15,30))
freqs = np.logspace( np.log10(2), np.log10(50), num=25)
n_cycles = 3
baseline = (-0.5, -0.1)
Fs = 1450.
decim = 14
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

epo_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/ave'
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/sensor'
df = read_csv(os.path.join(root_dir, 'afMSIT_sensor_info.csv'))

for ch in pick_channels:

    for analysis in ['stim', 'resp']:

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Iteratively load and merge.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        trials = []
        for subject in subjects:

            ## Load in epochs.
            epo_file = os.path.join(epo_dir, '%s_%s_%s_%s-epo.fif' %(subject,task,h_freq,analysis))
            epochs = read_epochs(epo_file, verbose=False)

            ## Get label index and extract.
            epochs.pick_channels([ch])
            trials.append(epochs._data.squeeze())

        ## Concatenate.
        trials = np.concatenate(trials, axis=0)
        times = epochs.times

        if not df.shape[0] == trials.shape[0]: raise ValueError('Incompatible dimensions!')
        
         #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Phase-lock removal.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
       
        for subject in df.Subject.unique():
            for dbs in [0, 1]:
                for cond in [0, 1]:
                    ix, = np.where((df.Subject==subject)&(df.DBS==dbs)&(df.Interference==cond))
                    trials[ix] -= trials[ix].mean(axis=0)
                    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute power.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
                    
        print 'Computing single trial power: %s %s.' %(ch, analysis)
        trials = np.expand_dims(trials,1)
        power = single_trial_power(trials, sfreq=Fs, frequencies=freqs, n_cycles=n_cycles, 
                                    baseline=None, n_jobs=n_jobs, verbose=False)
        power = power.squeeze()
        del trials
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Within-trial normalization.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        power = (power.T / np.median(power, axis=-1).T).T # median, not mean
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Prepare baseline normalization (within subject, DBS).
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Compute baseline normalization.
        if analysis == 'stim':

            ## Make time mask.
            mask = (times >= baseline[0]) & (times <= baseline[1])
            
            ## Iteratively compute over subjects.
            blnorm = []
            for subject in df.Subject.unique():
                
                ## Iteratively compute over DBS conditions..
                sbl = []
                for dbs in [0,1]:

                    ix, = np.where((df.Subject==subject)&(df.DBS==dbs))
                    sbl.append( np.apply_over_axes(np.median, power[ix][:,:,mask], axes=[0,2]).squeeze() )
            
                blnorm.append(sbl)
            
            ## Merge.
            blnorm = np.array(blnorm)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Apply baseline normalization (within subject, DBS).
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Setup index vectors.
        n_trials, n_freqs, n_times = power.shape        
        _, subj_ix = np.unique(df.Subject, return_inverse=True)
        dbs_ix = df.DBS.as_matrix()
        trial_ix = np.arange(n_trials) 
        
        ## Main loop.
        for i,j,k in zip(trial_ix,subj_ix,dbs_ix):
            
            for m in xrange(n_times):
                
                power[i,:,m] /= blnorm[j,k,:]
                
        if analysis == 'resp': del blnorm

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Final preprocessing steps.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        power = np.log10(power) * 10
            
        ## Crop times.
        if analysis == 'stim': mask = (times >= -0.5) & (times <= 2.0)
        elif analysis == 'resp': mask = (times >= -1.0) & (times <= 1.0)
        power = power[:, :, mask]
        times = times[mask]
            
        ## Iteratively average, transform, and save.
        for k,v in fdict.iteritems():
            
            ## Average across frequencies.
            ix, = np.where((freqs>=v[0])&(freqs<=v[1]))
            band = power[:,ix,:].mean(axis=1)
        
            ## Save.
            f = os.path.join(root_dir, 'afMSIT_sensor_%s_%s_%s' %(analysis, ch, k))
            np.savez_compressed(f, data=band[:,::decim], times=times[::decim], freqs=freqs, n_cycles=n_cycles)
        
        del power
        
print 'Done.'

### Perform permutatations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/afMSIT_permutations.py

### Identify clusters, perform FDR corrections, and assemble results.

In [ ]:
import os
import numpy as np
from collections import defaultdict
from mne.stats import fdr_correction
from pandas import DataFrame, concat
from scipy.ndimage import measurements
from scipy.stats import norm

def largest_cluster(arr, threshold):
    masked = np.abs( arr ) > threshold
    clusters, n_clusters = measurements.label(masked)
    cluster_sums = measurements.sum(arr, clusters, index=np.arange(n_clusters)+1)
    if not len(cluster_sums): return 0
    else: return np.abs(cluster_sums).max()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
labels = ['FCZ']
analysis = 'resp'
model_name = 'revised'
freqs = ['theta','alpha','beta']
domain = ['timedomain', 'frequency'][1]

## Statistics parameters.
alpha = 0.05
min_cluster = 0.05 # seconds

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initial preparations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define contrasts.
if model_name == 'revised': cols = ['Intercept', 'DBS', 'Interference', 'DBSxInt', 'nsArousal', 'nsValence', 'Trial']

## Define threshold.
threshold = norm.ppf(1 - alpha/2.)

## Read in seeds.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/seeds.txt'
with open(f, 'r') as f: seeds = [s.strip() for s in f.readlines()]
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#    

df = []
    
for label in labels:
    
    for freq in freqs:
    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Load files. Assemble permutations.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT/sensor'
        results_dir = os.path.join(root_dir, model_name)
        out_dir = os.path.join(root_dir, 'results')

        ## Load true statistics.
        npz = np.load(os.path.join(results_dir, 'afMSIT_sensor_%s_%s_%s_obs.npz' %(analysis, label, freq)))
        t_scores = npz['t_scores'].squeeze()
        times = npz['times'].squeeze()

        ## Load null statistics.
        for n, seed in enumerate(seeds):
            npz = np.load(os.path.join(results_dir, 'afMSIT_sensor_%s_%s_%s_%s.npz' %(analysis, label, freq, seed)))
            if not n: permuted = npz['t_scores']
            else: permuted = np.concatenate([permuted, npz['t_scores']], axis=0)
                
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute cluster statistics.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Get info.
        n_shuffles, n_eff, n_times  = permuted.shape

        ## Iteratively compute clusters.
        results = defaultdict(list)

        for n, con in enumerate(cols):

            ## Find real clusters.
            masked = np.abs( t_scores[n] ) > threshold
            clusters, n_clusters = measurements.label(masked)
            cluster_sums = measurements.sum(t_scores[n], clusters, index=np.arange(n_clusters)+1)

            ## Compute null cluster sums.
            null_sums = np.array([largest_cluster(permuted[m, n, :], threshold) for m in xrange(n_shuffles)])

            ## Compute cluster bounds.
            tmin = np.array([times[clusters==i].min() for i in np.arange(n_clusters)+1])
            tmax = np.array([times[clusters==i].max() for i in np.arange(n_clusters)+1])

            ## Find proportion of clusters that are larger.
            p_values = [(np.abs(cs) < null_sums).mean() for cs in cluster_sums]

            ## Store results.
            for t1, t2, cs, pval in zip(tmin,tmax,cluster_sums,p_values):
                results['Freq'] += [freq]
                results['Label'] += [label]
                results['Contrast'] += [con]
                results['Tmin'] += [t1]
                results['Tmax'] += [t2]
                results['Score'] += [cs]
                results['Pval'] += [pval]

        ## Organize results and append.
        results = DataFrame(results)
        results['Tdiff'] = results['Tmax'] - results['Tmin']
        results = results[results['Tdiff']>=min_cluster]
        df.append(results)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute cluster statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge dataframes.
df = concat(df)

## Organize columns and sort.
cols = ['Contrast','Label','Freq','Tmin','Tmax','Tdiff','Score','Pval']
df = df[cols].sort_values(['Contrast','Tmin']).reset_index(drop=True)

## FDR correct within contrasts.
df['FDR'] = 0
for contrast in df.Contrast.unique():
    _, fdr = fdr_correction(df.loc[df.Contrast==contrast,'Pval'], alpha)
    df.loc[df.Contrast==contrast,'FDR'] = fdr
    
## Save.
f = os.path.join(out_dir, '%s_%s_%s_results.csv' %(model_name, analysis,domain))
df.to_csv(f, index=False)
        
print 'Done.'

## Source Space Analysis
### Source localize single trial epochs

In [ ]:
import os
import numpy as np
import pylab as plt
from mne import read_epochs, read_label, read_source_spaces, set_log_level
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from scipy.io import loadmat
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
analysis = 'resp'
parc = 'april2016'
fmax = 50

## Source localization parameters.
method = 'dSPM'
snr = 1.0  
lambda2 = 1.0 / snr ** 2
pick_ori = 'normal'

## Labels
rois = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Iteratively load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'

## Prepare fsaverage source space.
src = read_source_spaces(os.path.join(fs_dir,'fscopy','bem','fscopy-oct-6p-src.fif'))
vertices_to = [src[n]['vertno'] for n in xrange(2)]
labels = [read_label(os.path.join(fs_dir,'fscopy','label','april2016','%s.label' %roi), subject='fsaverage')
          for roi in rois]

for subject in subjects:

    print 'Performing source localization: %s' %subject

    ## Load in epochs.
    epochs = read_epochs(os.path.join(root_dir,'ave','%s_%s_%s_%s-epo.fif' %(subject,task,fmax,analysis)))
    times = epochs.times
    
    ## Load in secondary files.
    inv = read_inverse_operator(os.path.join(root_dir,'cov','%s_%s_%s-inv.fif' %(subject,task,fmax)))
    morph_mat = loadmat(os.path.join(root_dir, 'morph_maps', '%s-fsaverage_morph.mat' %subject))['morph_mat']

    ## Make generator object.
    G = apply_inverse_epochs(epochs, inv, method=method, lambda2=lambda2, pick_ori=pick_ori, return_generator=True)
    del epochs, inv

    ## Iteratively compute and store label timecourse. 
    ltcs = []
    for g in G:
        g = g.morph_precomputed('fsaverage', vertices_to=vertices_to, morph_mat=morph_mat)
        ltcs.append( g.extract_label_time_course(labels, src, mode='pca_flip') )
    ltcs = np.array(ltcs)
    
    ## Save.
    f = os.path.join(root_dir,'source','stcs','%s_%s_%s_%s_%s_epochs' %(subject,task,analysis,method,fmax))
    np.savez_compressed(f, ltcs=ltcs, times=times, labels=np.array([l.name for l in labels]))
    del ltcs
    
print 'Done.'

### Reassemble source localized epochs

In [ ]:
import os
import numpy as np
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
method = 'dSPM'
h_freq = 50

## Label parameters.
rois = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

## Processing parameters.
fmax = 15
sfreq = 1450.
decim = 3
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'
df = read_csv(os.path.join(root_dir, 'afMSIT_source_info.csv'))

for analysis in ['stim', 'resp']:

    for roi in rois:

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Iteratively load and merge.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ltcs = []
        for subject in subjects:

            ## Load NPZ.
            npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_%s_%s_%s_epochs.npz' %(subject,analysis,method,h_freq)))

            ## Get label index and extract.
            ix = npz['labels'].tolist().index(roi)
            arr = npz['ltcs'][:,ix,:]

            ## Append.
            ltcs.append(arr)

        ## Concatenate.
        ltcs = np.concatenate(ltcs, axis=0)
        times = npz['times']

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Time-domain processing.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Make ERP objects.
        ERPs = low_pass_filter(ltcs, sfreq, fmax, filter_length='20s', n_jobs=n_jobs)

        ## Crop times.
        if analysis == 'stim': mask = (times >= -0.5) & (times <= 2.0)
        elif analysis == 'resp': mask = (times >= -1.0) & (times <= 1.0)
        ERPs = ERPs[:, mask]
        times = times[mask]
        
        ## Save.
        np.savez_compressed(os.path.join(root_dir, 'afMSIT_source_%s_%s_%s' %(analysis,roi,fmax)), 
                            data=ERPs[:,::decim], times=times[::decim], method=method)
        
print 'Done.'

### Compute power of source localized epochs

In [ ]:
import os
import numpy as np
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
method = 'dSPM'
h_freq = 50

## Label parameters.
rois = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

## Time-frequency parameters.
fdict = dict(theta = (4,8), alpha = (8,15), beta = (15,30))
freqs = np.logspace( np.log10(2), np.log10(50), num=25)
n_cycles = 3
baseline = (-0.5, -0.1)
Fs = 1450.
decim = 14
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'
df = read_csv(os.path.join(root_dir, 'afMSIT_source_info.csv'))

for roi in rois:

    for analysis in ['stim', 'resp']:

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Iteratively load and merge.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ltcs = []
        for subject in subjects:

            ## Load NPZ.
            npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_%s_%s_%s_epochs.npz' %(subject,analysis,method,h_freq)))

            ## Get label index and extract.
            ix = npz['labels'].tolist().index(roi)
            arr = npz['ltcs'][:,ix,:]

            ## Append.
            ltcs.append(arr)

        ## Concatenate.
        ltcs = np.concatenate(ltcs, axis=0)
        times = npz['times']

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Phase-lock removal.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
       
        for subject in df.Subject.unique():
            for dbs in [0, 1]:
                for cond in [0, 1]:
                    ix, = np.where((df.Subject==subject)&(df.DBS==dbs)&(df.Interference==cond))
                    ltcs[ix] -= ltcs[ix].mean(axis=0)
                    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute power.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
                    
        print 'Computing single trial power: %s %s.' %(roi, analysis)
        ltcs = np.expand_dims(ltcs,1)
        power = single_trial_power(ltcs, sfreq=Fs, frequencies=freqs, n_cycles=n_cycles, 
                                    baseline=None, n_jobs=n_jobs, verbose=False)
        power = power.squeeze()
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Within-trial normalization.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        power = (power.T / np.median(power, axis=-1).T).T # median, not mean
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Prepare baseline normalization (within subject, DBS).
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Compute baseline normalization.
        if analysis == 'stim':

            ## Make time mask.
            mask = (times >= baseline[0]) & (times <= baseline[1])
            
            ## Iteratively compute over subjects.
            blnorm = []
            for subject in df.Subject.unique():
                
                ## Iteratively compute over DBS conditions..
                sbl = []
                for dbs in [0,1]:

                    ix, = np.where((df.Subject==subject)&(df.DBS==dbs))
                    sbl.append( np.apply_over_axes(np.median, power[ix][:,:,mask], axes=[0,2]).squeeze() )
            
                blnorm.append(sbl)
            
            ## Merge.
            blnorm = np.array(blnorm)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Apply baseline normalization (within subject, DBS).
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Setup index vectors.
        n_trials, n_freqs, n_times = power.shape        
        _, subj_ix = np.unique(df.Subject, return_inverse=True)
        dbs_ix = df.DBS.as_matrix()
        trial_ix = np.arange(n_trials) 
        
        ## Main loop.
        for i,j,k in zip(trial_ix,subj_ix,dbs_ix):
            
            for m in xrange(n_times):
                
                power[i,:,m] /= blnorm[j,k,:]
                
        if analysis == 'resp': del blnorm

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Final preprocessing steps.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            
        ## Convert to decibels.
        power = np.log10(power) * 10
            
        ## Crop times.
        if analysis == 'stim': mask = (times >= -0.5) & (times <= 2.0)
        elif analysis == 'resp': mask = (times >= -1.0) & (times <= 1.0)
        power = power[:, :, mask]
        times = times[mask]
            
        ## Iteratively average, transform, and save.
        for k,v in fdict.iteritems():
            
            ## Average across frequencies.
            ix, = np.where((freqs>=v[0])&(freqs<=v[1]))
            band = power[:,ix,:].mean(axis=1)
        
            ## Save.
            f = os.path.join(root_dir, 'afMSIT_source_%s_%s_%s' %(analysis, roi, k))
            np.savez_compressed(f, data=band[:,::decim], times=times[::decim], freqs=freqs, n_cycles=n_cycles)
        
        del power
        
print 'Done.'

### Perform permutatations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/afMSIT_permutations.py

### Identify clusters, perform FDR corrections, and assemble results.

In [ ]:
import os
import numpy as np
from collections import defaultdict
from mne.stats import fdr_correction
from pandas import DataFrame, concat
from scipy.ndimage import measurements
from scipy.stats import norm

def largest_cluster(arr, threshold):
    masked = np.abs( arr ) > threshold
    clusters, n_clusters = measurements.label(masked)
    cluster_sums = measurements.sum(arr, clusters, index=np.arange(n_clusters)+1)
    if not len(cluster_sums): return 0
    else: return np.abs(cluster_sums).max()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
labels = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
          'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
          'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

analysis = 'resp'
model_name = 'revised'
freqs = ['theta', 'alpha', 'beta']
domain = ['timedomain', 'frequency'][1]

## Statistics parameters.
alpha = 0.05
min_cluster = 0.05 # seconds

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initial preparations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define contrasts.
if model_name == 'revised': cols = ['Intercept', 'DBS', 'Interference', 'DBSxInt', 'nsArousal', 'nsValence', 'Trial']

## Define threshold.
threshold = norm.ppf(1 - alpha/2.)

## Read in seeds.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/seeds.txt'
with open(f, 'r') as f: seeds = [s.strip() for s in f.readlines()]
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#    

df = []
    
for label in labels:
    
    for freq in freqs:
    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Load files. Assemble permutations.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT/source'
        results_dir = os.path.join(root_dir, model_name)
        out_dir = os.path.join(root_dir, 'results')

        ## Load true statistics.
        npz = np.load(os.path.join(results_dir, 'afMSIT_source_%s_%s_%s_obs.npz' %(analysis, label, freq)))
        t_scores = npz['t_scores'].squeeze()
        times = npz['times'].squeeze()

        ## Load null statistics.
        for n, seed in enumerate(seeds):
            npz = np.load(os.path.join(results_dir, 'afMSIT_source_%s_%s_%s_%s.npz' %(analysis, label, freq, seed)))
            if not n: permuted = npz['t_scores']
            else: permuted = np.concatenate([permuted, npz['t_scores']], axis=0)
                
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute cluster statistics.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Get info.
        n_shuffles, n_eff, n_times  = permuted.shape

        ## Iteratively compute clusters.
        results = defaultdict(list)

        for n, con in enumerate(cols):

            ## Find real clusters.
            masked = np.abs( t_scores[n] ) > threshold
            clusters, n_clusters = measurements.label(masked)
            cluster_sums = measurements.sum(t_scores[n], clusters, index=np.arange(n_clusters)+1)

            ## Compute null cluster sums.
            null_sums = np.array([largest_cluster(permuted[m, n, :], threshold) for m in xrange(n_shuffles)])

            ## Compute cluster bounds.
            tmin = np.array([times[clusters==i].min() for i in np.arange(n_clusters)+1])
            tmax = np.array([times[clusters==i].max() for i in np.arange(n_clusters)+1])

            ## Find proportion of clusters that are larger.
            p_values = [((np.abs(cs) < null_sums).sum() + 1.) / (null_sums.shape[0] + 1.) for cs in cluster_sums]
            
            ## Store results.
            for t1, t2, cs, pval in zip(tmin,tmax,cluster_sums,p_values):
                results['Freq'] += [freq]
                results['Label'] += [label]
                results['Contrast'] += [con]
                results['Tmin'] += [t1]
                results['Tmax'] += [t2]
                results['Score'] += [cs]
                results['Pval'] += [pval]

        ## Organize results and append.
        results = DataFrame(results)
        results['Tdiff'] = results['Tmax'] - results['Tmin']
        results = results[results['Tdiff']>=min_cluster]
        df.append(results)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute cluster statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge dataframes.
df = concat(df)

## Organize columns and sort.
cols = ['Contrast','Label','Freq','Tmin','Tmax','Tdiff','Score','Pval']
df = df[cols].sort_values(['Contrast','Tmin']).reset_index(drop=True)

## FDR correct within contrasts.
df['FDR'] = 0
for contrast in df.Contrast.unique():
    _, fdr = fdr_correction(df.loc[df.Contrast==contrast,'Pval'], alpha)
    df.loc[df.Contrast==contrast,'FDR'] = fdr
    
## Save.
f = os.path.join(out_dir, '%s_%s_%s_results.csv' %(model_name, analysis,domain))
df.to_csv(f, index=False)
        
print 'Done.'

# Section 3: Secondary Analyses

## Individual Regressions on DLPFC-5 LH 
Here we look for the fraction of subjects that show the same DBSon/DBSoff effects in DLPFC-5 LH at the single-subject level.
### Perform permutations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/subject_permutations.py
### Identify clusters, perform FDR corrections, and assemble results.

In [ ]:
import os
import numpy as np
from collections import defaultdict
from mne.stats import fdr_correction
from pandas import DataFrame, concat
from scipy.ndimage import measurements
from scipy.stats import norm

def largest_cluster(arr, threshold):
    masked = np.abs( arr ) > threshold
    clusters, n_clusters = measurements.label(masked)
    cluster_sums = measurements.sum(arr, clusters, index=np.arange(n_clusters)+1)
    if not len(cluster_sums): return 0
    else: return np.abs(cluster_sums).max()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JASE', 'M5', 'MEWA', 'S2']
labels = ['dlpfc_5-lh']

space = 'source'
analysis = 'stim'
model_name = 'revised'
freqs = ['theta']
domain = ['timedomain', 'frequency'][1]

## Statistics parameters.
alpha = 0.05
min_cluster = 0.05 # seconds

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initial preparations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define contrasts.
if model_name == 'revised': cols = ['Intercept', 'DBS', 'Interference', 'DBSxInt', 'nsArousal', 'nsValence', 'Trial']

## Define threshold.
threshold = norm.ppf(1 - alpha/2.)

## Read in seeds.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/seeds.txt'
with open(f, 'r') as f: seeds = [s.strip() for s in f.readlines()]
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#    

df = []
    
for subject in subjects: 
        
    for label in labels:

        for freq in freqs:

            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Load files. Assemble permutations.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT/%s' %space
            results_dir = os.path.join(root_dir, model_name)
            out_dir = os.path.join(root_dir, 'results')

            ## Load true statistics.
            npz = np.load(os.path.join(results_dir, '%s_%s_%s_%s_%s_obs.npz' %(subject, space, analysis, label, freq)))
            t_scores = npz['t_scores'].squeeze()
            times = npz['times'].squeeze()

            ## Load null statistics.
            for n, seed in enumerate(seeds):
                npz = np.load(os.path.join(results_dir, '%s_%s_%s_%s_%s_%s.npz' %(subject, space, analysis, label, freq, seed)))
                if not n: permuted = npz['t_scores']
                else: permuted = np.concatenate([permuted, npz['t_scores']], axis=0)

            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Compute cluster statistics.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

            ## Get info.
            n_shuffles, n_eff, n_times  = permuted.shape

            ## Iteratively compute clusters.
            results = defaultdict(list)

            for n, con in enumerate(cols):

                ## Find real clusters.
                masked = np.abs( t_scores[n] ) > threshold
                clusters, n_clusters = measurements.label(masked)
                cluster_sums = measurements.sum(t_scores[n], clusters, index=np.arange(n_clusters)+1)

                ## Compute null cluster sums.
                null_sums = np.array([largest_cluster(permuted[m, n, :], threshold) for m in xrange(n_shuffles)])

                ## Compute cluster bounds.
                tmin = np.array([times[clusters==i].min() for i in np.arange(n_clusters)+1])
                tmax = np.array([times[clusters==i].max() for i in np.arange(n_clusters)+1])

                ## Find proportion of clusters that are larger.
                p_values = [((np.abs(cs) < null_sums).sum() + 1.) / (null_sums.shape[0] + 1.) for cs in cluster_sums]

                ## Store results.
                for t1, t2, cs, pval in zip(tmin,tmax,cluster_sums,p_values):
                    results['Subject'] += [subject]
                    results['Freq'] += [freq]
                    results['Label'] += [label]
                    results['Contrast'] += [con]
                    results['Tmin'] += [t1]
                    results['Tmax'] += [t2]
                    results['Score'] += [cs]
                    results['Pval'] += [pval]

            ## Organize results and append.
            results = DataFrame(results)
            results['Tdiff'] = results['Tmax'] - results['Tmin']
            results = results[results['Tdiff']>=min_cluster]
            df.append(results)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute cluster statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge dataframes.
df = concat(df)

## Organize columns and sort.
cols = ['Subject','Contrast','Label','Freq','Tmin','Tmax','Tdiff','Score','Pval']
df = df[cols].sort_values(['Subject','Contrast','Tmin']).reset_index(drop=True)

## FDR correct within contrasts.
df['FDR'] = 0
for contrast in df.Contrast.unique():
    _, fdr = fdr_correction(df.loc[df.Contrast==contrast,'Pval'], alpha)
    df.loc[df.Contrast==contrast,'FDR'] = fdr
    
## Save.
f = os.path.join(out_dir, 'subjects_%s_%s_%s_results.csv' %(model_name, analysis, domain))
df.to_csv(f, index=False)
        
print 'Done.'

df[(df.Contrast=='DBS')&(df.FDR<0.05)]

## Reaction Time Correlations
### Perform permutations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/afMSIT_permutations.py
### Identify clusters, perform FDR corrections, and assemble results.

In [4]:
import os
import numpy as np
from pandas import concat, read_csv
from mne.stats import fdr_correction

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O Parameters.
space = 'source'
analysis = 'resp'
labels = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
          'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
          'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']
domain = 'timedomain'
freqs = ['15']
model = 'revised'

## Cluster parameters.
min_cluster = 0.05 # secs
alpha = 0.05

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s/rt' %space
results_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s/results' %space

df = []
for label in labels:
    
    for freq in freqs:
        
        df.append( read_csv(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s_rt.csv' %(space, analysis, label, freq))) )

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Merge, trim, correct, and save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
## Merge and trim.
df = concat(df)
df = df[df.Tdiff > min_cluster]

## FDR correct.
df['FDR'] = fdr_correction(df.Pval, alpha=alpha)[-1]

## Save.
f = os.path.join(results_dir, '%s_%s_%s_rt.csv' %(model, analysis, domain))
df.to_csv(f, index=False)
print 'Done.'

Done.


## MADRS Correlations
### Permutations for MADRS Correlations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/afMSIT_permutations.py

In [ ]:
import os
import numpy as np
from pandas import DataFrame
np.random.seed(47404)

## Define parameters.
n_subj = 8
n_shuffles = 1000

## Prepare
ix = np.arange(n_subj)
permutations = np.expand_dims(ix[::-1], 0)
    
while True: 
    np.random.shuffle(ix)
    permutations = np.concatenate([permutations, np.expand_dims(ix, 0)], axis=0)
    permutations = DataFrame(permutations).drop_duplicates()
    if permutations.shape[0] < n_shuffles: permutations = permutations.as_matrix()
    else: break
        
## Save.
np.save('/space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/madrs_permutations', permutations.as_matrix())

### MADRS Correlations

In [ ]:
import os
import numpy as np
from pandas import read_csv
from scipy.stats import spearmanr
from mne.stats import fdr_correction

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

space = 'source'
analysis = 'stim'
domain = 'frequency'
model = 'revised'
contrast = 'DBS'
fdr = 0.05

subjects = np.array(['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA'])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare MADRS Scores.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

## Prepare MADRS scores.
ratings = read_csv('/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior/Subject_Rating_Scales.csv')
ratings = ratings.set_index('Subject')
madrs = ratings.loc[subjects, 'MADRS_Now'] - ratings.loc[subjects, 'MADRS_Base']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

results = read_csv(os.path.join(root_dir, 'results', '%s_%s_%s_results.csv' %(model,analysis,domain)))
results = results[(results.Contrast==contrast)&(results.FDR<fdr)].reset_index(drop=True)

results['Corr_R'] = 0 
results['Corr_P'] = 0 

for n in xrange(results.shape[0]):
    
    label = results.loc[n, 'Label']
    freq  = results.loc[n, 'Freq']
    tmin  = results.loc[n, 'Tmin']
    tmax  = results.loc[n, 'Tmax']
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space, analysis, label, freq)))
    data = npz['data']
    times = npz['times']

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute condition differences.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    delta = np.zeros(subjects.shape[0])
    mask = (times >= tmin) & (times <= tmax)    
    
    for m, subject in enumerate(subjects):
        i, = np.where((info['Subject']==subject)&(info[contrast]==0))
        j, = np.where((info['Subject']==subject)&(info[contrast]==1))
        delta[m] += (data[j][:,mask].mean(axis=0) - data[i][:,mask].mean(axis=0)).mean()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform correlations.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Compute true correlations.
    r, p = spearmanr(madrs, delta)
    if label == 'dlpfc_5-lh': test = delta.copy()
    results.loc[n, 'Corr_R'] = r
    results.loc[n, 'Corr_P'] = p

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Save.
results['Corr_FDR'] = fdr_correction(results.Corr_P)[-1]
results = results.sort_values('Corr_FDR')

f = os.path.join(root_dir, 'results', '%s_%s_%s_madrs.csv' %(model, analysis, domain))
results.to_csv(f, index=False)
    
print 'Done.'

### Special MADRS Correlation for DLPFC_5-LH (Stimulus-Locked)

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv
from scipy.stats import spearmanr
from mne.stats import fdr_correction
np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
space = 'source'
analysis = 'stim'
domain = 'frequency'
model = 'revised'
contrast = 'DBS'
fdr = 0.05

## Subject parameters.
subjects = np.array(['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2'])

## Bootstrapping parameters.
n_straps = 1000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig = plt.figure(figsize=(12,12))
true_scores = np.zeros((2,2))
bootstraps = np.zeros((2,n_straps))

## Looping over including and excluding S2.
for n in np.arange(2): 

    if n: subjects = np.delete(subjects, -1)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Prepare MADRS Scores.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
    info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

    ## Prepare MADRS scores.
    ratings = read_csv('/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior/Subject_Rating_Scales.csv')
    ratings = ratings.set_index('Subject')
    madrs = ratings.loc[subjects, 'MADRS_Now'] - ratings.loc[subjects, 'MADRS_Base']

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load cluster results.
    results = read_csv(os.path.join(root_dir, 'results', '%s_%s_%s_results.csv' %(model,analysis,domain)))
    results = results[(results.Contrast==contrast)&(results.FDR<fdr)].reset_index(drop=True)
    results = results[results.Label == 'dlpfc_5-lh'] # Limit to the theta clusters.

    ## Load time series data.
    npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space, analysis, 'dlpfc_5-lh', 'theta')))
    data = npz['data']
    times = npz['times']

    ## Compute condition differences.
    delta = np.zeros(subjects.shape[0])
    mask = (times >= results.Tmin.min()) & (times <= results.Tmax.min())    

    for m, subject in enumerate(subjects):
        i, = np.where((info['Subject']==subject)&(info[contrast]==0))
        j, = np.where((info['Subject']==subject)&(info[contrast]==1))
        delta[m] += (data[j][:,mask].mean(axis=0) - data[i][:,mask].mean(axis=0)).mean()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform correlations.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Compute true correlations.
    true_scores[n] = spearmanr(madrs, delta)
    
    ## Perform bootstrapping
    n_subj = subjects.shape[0]
    for m in np.arange(n_straps):
        
        ix = np.random.choice(np.arange(n_subj), n_subj, replace=True)
        bootstraps[n,m], _ = spearmanr(madrs[ix], delta[ix])
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plot Scatterplot.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ax = plt.subplot2grid((2,2),(0,n))
    if not n: color = '#2b8cbe'
    else: color = '#31a354'

    ## Iteratively add points.
    for x,y,subj in zip(madrs,delta,madrs.index):

        ax.scatter(x,y,s=80,color=color)
        ax.text(x+0.5,y+0.05,subj)

    ## Add trend line.
    x1, x2 = ax.get_xlim()
    y1, y2 = ax.get_ylim()

    z = np.polyfit(madrs, delta, 1)
    p = np.poly1d(z)
    ax.plot(np.linspace(x1,x2,1e3), p(np.linspace(x1,x2,1e3)), linewidth=2, color='k', alpha=0.8)
    ax.set_xlim(x1,x2)
    ax.set_ylim(y1,y2)

    ## Add flourishes.
    ax.set_xlabel('MADRS [Post - Pre]', fontsize=18)
    ax.set_ylabel('Theta Power [On - Off]', fontsize=18)
    if not n: ax.set_title('DLPFC-5 LH (S2 included)', fontsize=24)
    else: ax.set_title('DLPFC-5 LH (S2 discarded)', fontsize=24)
    ax.text(x1+0.5, y2*0.99, 'r = %0.3f\np = %0.3f' %(true_scores[n,0],true_scores[n,1]), 
            fontsize=18, va='top')

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plot Histogram.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ax = plt.subplot2grid((2,2),(1,n))

    ## Plot histogram.
    x = bootstraps[n]
    x = x[~np.isnan(x)]
    ax.hist(x, bins=40, color=color)

    ## Add true score.
    y1, y2 = ax.get_ylim()
    ax.vlines(true_scores[n,0], y1, y2, linestyle='--', linewidth=2)
    ax.text(true_scores[n,0], y2*0.98, 'r = %0.3f' %true_scores[n,0], 
            ha='right', va='top', rotation='vertical', weight='heavy')
    
    ## Add flourishes.
    ax.set_xlabel('Spearman R', fontsize=18) 
    ax.set_ylabel('Frequency', fontsize=18)
    if not n: ax.set_title('%s Bootstraps (S2 included)' %n_straps, fontsize=20)
    else: ax.set_title('%s Bootstraps (S2 discarded)' %n_straps, fontsize=20)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#        

plt.tight_layout()
plt.show()

### Special Hypomania Correlation for DLPFC_5-LH (Stimulus-Locked)

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv
from scipy.stats import spearmanr
from mne.stats import fdr_correction
np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
space = 'source'
analysis = 'stim'
domain = 'frequency'
model = 'revised'
contrast = 'DBS'
fdr = 0.05

## Subject parameters.
subjects = np.array(['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA'])

## Bootstrapping parameters.
n_straps = 1000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare MADRS Scores.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

## Prepare MADRS scores.
ratings = read_csv('/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior/Subject_Characteristics.csv')
ratings = ratings.set_index('Name')
mania = ratings.loc[subjects, 'Hypomania']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load cluster results.
results = read_csv(os.path.join(root_dir, 'results', '%s_%s_%s_results.csv' %(model,analysis,domain)))
results = results[(results.Contrast==contrast)&(results.FDR<fdr)].reset_index(drop=True)
results = results[results.Label == 'dlpfc_5-lh'] # Limit to the theta clusters.

## Load time series data.
npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space, analysis, 'dlpfc_5-lh', 'theta')))
data = npz['data']
times = npz['times']

## Compute condition differences.
delta = np.zeros(subjects.shape[0])
mask = (times >= results.Tmin.min()) & (times <= results.Tmax.min())    

for m, subject in enumerate(subjects):
    i, = np.where((info['Subject']==subject)&(info[contrast]==0))
    j, = np.where((info['Subject']==subject)&(info[contrast]==1))
    delta[m] += (data[j][:,mask].mean(axis=0) - data[i][:,mask].mean(axis=0)).mean()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Perform correlations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Compute true correlations.
true_scores = spearmanr(mania, delta)

## Perform bootstrapping
n_subj = subjects.shape[0]
bootstraps = np.zeros(n_straps)

for m in np.arange(n_straps):

    ix = np.random.choice(np.arange(n_subj), n_subj, replace=True)
    bootstraps[m], _ = spearmanr(mania[ix], delta[ix])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot Scatterplot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig = plt.figure(figsize=(12,6))
ax = plt.subplot2grid((1,2),(0,0))
color = '#2b8cbe'

## Iteratively add points.
for x,y,subj in zip(mania,delta,mania.index):

    ax.scatter(x,y,s=80,color=color)
    ax.text(x+0.01,y+0.05,subj)

## Add trend line.
x1, x2 = ax.get_xlim()
y1, y2 = ax.get_ylim()

z = np.polyfit(mania, delta, 1)
p = np.poly1d(z)
ax.plot(np.linspace(x1,x2,1e3), p(np.linspace(x1,x2,1e3)), linewidth=2, color='k', alpha=0.8)
ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)

## Add flourishes.
ax.set_xlabel('Hypomania', fontsize=18)
ax.set_ylabel('Theta Power [On - Off]', fontsize=18)
ax.set_title('DLPFC-5 LH', fontsize=24)
ax.text(x1+0.01, y2*0.99, 'r = %0.3f\np = %0.3f' %(true_scores[0],true_scores[1]), 
        fontsize=18, va='top')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot Histogram.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

ax = plt.subplot2grid((1,2),(0,1))

## Plot histogram.
x = bootstraps[~np.isnan(bootstraps)]
ax.hist(x, bins=40, color=color)

## Add true score.
y1, y2 = ax.get_ylim()
ax.vlines(true_scores[0], y1, y2, linestyle='--', linewidth=2)
ax.text(true_scores[0], y2*0.98, 'r = %0.3f' %true_scores[0], 
        ha='right', va='top', rotation='vertical', weight='heavy')

## Add flourishes.
ax.set_xlabel('Spearman R', fontsize=18) 
ax.set_ylabel('Frequency', fontsize=18)
ax.set_title('%s Bootstraps' %n_straps, fontsize=20)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#        

plt.tight_layout()
plt.show()

## Intertrial Interval Spectra Comparison
### Make ITI epochs
Each trial is 1.93s. On average 5.6s between each trial. Minimum time between trials is 3.0s between trials. We take 2s windows of time, centered at the time between trials, using the baseline for the proceeding trial.

In [ ]:
import os
import numpy as np
from mne import compute_covariance, Epochs, EpochsArray, find_events, read_proj, pick_types, set_log_level
from mne.io import Raw
from pandas import read_csv
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'

## Filtering parameters.
l_freq = 0.5
h_freq = 50
l_trans_bandwidth = l_freq / 2.
h_trans_bandwidth = 1.0
filter_length = '20s'
n_jobs = 3

## Epoching parameters.
event_id = dict( FN=1, FI=2, NN=3, NI=4 )      # Alik's convention, isn't he smart!?
tmin = -1.25
tmax = 1.25
baseline = (-0.5, -0.1)
reject   = dict(eeg=150e-6)
flat     = dict(eeg=5e-7)
detrend = None
decim = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load behavior.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT'
data_file = os.path.join( root_dir, 'behavior', 'afMSIT_group_data.csv' )
df = read_csv(data_file)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for subj in subjects:
    
    print 'Loading data for %s.' %subj

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    # Define paths.
    root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT'
    raw_file  = os.path.join( root_dir, 'raw', '%s_%s_raw.fif' %(subj,task) )
    proj_file = os.path.join( root_dir, 'raw', '%s_%s-proj.fif' %(subj,task) )
    
    # Load data.
    raw = Raw(raw_file,add_eeg_ref=False,preload=True,verbose=False)
    proj = read_proj(proj_file)
    
    ## Add projections.
    proj = [p for p in proj if 'ref' not in p['desc']]
    raw.add_proj(proj, remove_existing=True)
    raw.set_eeg_reference()
    raw.apply_proj()
    
    ## Reduce dataframe to subject.
    data = df[df.Subject==subj]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make events.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    trial_onsets  = find_events(raw, stim_channel='Trig1', output='onset', min_duration=0.25, verbose=False)
    trial_offsets = find_events(raw, stim_channel='Trig2', output='offset', min_duration=0.25, verbose=False)
    
    ## Identify time at halfway point between end of trial(n-1) and start of trial(n).
    iti_lengths = raw.times[trial_onsets[1:,0]] - raw.times[trial_offsets[:-1,0]]
    iti_centers = raw.times[trial_offsets[:-1,0]] + iti_lengths / 2.

    ## Prepend first ITI center as 1.5 prior to first onset (corresponding to a 3s "ITI" before first trial).
    first_onset = raw.times[trial_onsets[0,0]]
    iti_centers = np.insert(iti_centers,0,first_onset - 1.5)

    ## Modify first event of DBS-on so that center of ITI prior to first onset occurs 1.5s before onset.
    ix = np.where( (data.DBS==1) & (data.Trial==1) )[0][0]
    first_onset = raw.times[trial_onsets[ix,0]]
    iti_centers[ix] = first_onset - 1.5

    ## Make events.
    events = np.zeros_like(trial_offsets)
    events.T[0] += raw.time_as_index(iti_centers)
    
    # Update event identifiers.
    n = 1
    for dbs in [0,1]:
        for cond in [0,1]:
            ix, = np.where((data.DBS==dbs)&(data.Interference==cond))
            events[ix,-1] = n
            n+=1
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Filter
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    Fs = raw.info['sfreq']
    raw.filter(l_freq = l_freq, h_freq = h_freq, filter_length=filter_length, n_jobs=n_jobs,
               l_trans_bandwidth=l_trans_bandwidth, h_trans_bandwidth=h_trans_bandwidth)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make epochs.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    # Make initial ITI epochs.
    picks = pick_types(raw.info, meg=False, eeg=True, exclude='bads')
    epochs = Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, 
                    baseline=None, picks=picks, reject=None, flat=None, proj=True, 
                    detrend=detrend, decim=decim)

    # Make baseline periods.
    bl = Epochs(raw, trial_onsets, tmin=baseline[0], tmax=baseline[1], 
                baseline=None, picks=picks, reject=None, flat=None, proj=True, 
                detrend=detrend, decim=decim)
    
    # Baseline correction.
    arr = epochs.get_data()
    bl = bl.get_data().mean(axis=-1)
    arr = (arr.T - bl.T).T
    
    # Finalize ITI epochs.
    epochs = EpochsArray(arr, epochs.info, events=epochs.events, tmin=epochs.tmin, 
                         baseline=None, reject=reject, flat=flat, 
                         proj=True, verbose=False)
    print epochs
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    epochs.save(os.path.join(root_dir,'ave','%s_%s_%s_iti-epo.fif' %(subj,task,h_freq)))
    
    print '\n#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#\n'
    
print 'Done.'

### Make source localized ITI epochs

In [ ]:
import os
import numpy as np
import pylab as plt
from mne import read_epochs, read_label, read_source_spaces, set_log_level
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from scipy.io import loadmat
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
analysis = 'iti_bl'
parc = 'april2016'
fmax = 50

## Source localization parameters.
method = 'dSPM'
snr = 1.0  
lambda2 = 1.0 / snr ** 2
pick_ori = 'normal'

## Labels
rois = ['dlpfc_5-lh']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Iteratively load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'

## Prepare fsaverage source space.
src = read_source_spaces(os.path.join(fs_dir,'fscopy','bem','fscopy-oct-6p-src.fif'))
vertices_to = [src[n]['vertno'] for n in xrange(2)]
labels = [read_label(os.path.join(fs_dir,'fscopy','label','april2016','%s.label' %roi), subject='fsaverage')
          for roi in rois]

for subject in subjects:

    print 'Performing source localization: %s' %subject

    ## Load in epochs.
    epochs = read_epochs(os.path.join(root_dir,'ave','%s_%s_%s_%s-epo.fif' %(subject,task,fmax,analysis)))
    times = epochs.times
    
    ## Load in secondary files.
    inv = read_inverse_operator(os.path.join(root_dir,'cov','%s_%s_%s-inv.fif' %(subject,task,fmax)))
    morph_mat = loadmat(os.path.join(root_dir, 'morph_maps', '%s-fsaverage_morph.mat' %subject))['morph_mat']

    ## Make generator object.
    G = apply_inverse_epochs(epochs, inv, method=method, lambda2=lambda2, pick_ori=pick_ori, return_generator=True)
    del inv

    ## Iteratively compute and store label timecourse. 
    ltcs = []
    for g in G:
        g = g.morph_precomputed('fsaverage', vertices_to=vertices_to, morph_mat=morph_mat)
        ltcs.append( g.extract_label_time_course(labels, src, mode='pca_flip') )
    ltcs = np.array(ltcs)
    
    ## Save.
    f = os.path.join(root_dir,'source','stcs','%s_%s_%s_%s_%s_epochs' %(subject,task,analysis,method,fmax))
    np.savez_compressed(f, ltcs=ltcs, times=times, labels=np.array([l.name for l in labels]), conds=epochs.events.T[-1])
    del ltcs
    
print 'Done.'

### Make power baseline for ITI spectral epochs

In [ ]:
import os
import numpy as np
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
analysis = 'stim'
method = 'dSPM'
h_freq = 50

## Label parameters.
rois = [ 'dlpfc_5-lh']

## Time-frequency parameters.
freqs = np.logspace( np.log10(2), np.log10(50), num=25)
n_cycles = 3
baseline = (-0.5, -0.1)
Fs = 1450.
decim = 14
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'
df = read_csv(os.path.join(root_dir, 'afMSIT_source_info.csv'))

for roi in rois:

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Iteratively load and merge.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ltcs = []
    for subject in subjects:

        ## Load NPZ.
        npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_%s_%s_%s_epochs.npz' %(subject,analysis,method,h_freq)))

        ## Get label index and extract.
        ix = npz['labels'].tolist().index(roi)
        arr = npz['ltcs'][:,ix,:]

        ## Append.
        ltcs.append(arr)

    ## Concatenate.
    ltcs = np.concatenate(ltcs, axis=0)
    times = npz['times']

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Phase-lock removal.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    for subject in df.Subject.unique():
        for dbs in [0, 1]:
            for cond in [0, 1]:
                ix, = np.where((df.Subject==subject)&(df.DBS==dbs)&(df.Interference==cond))
                ltcs[ix] -= ltcs[ix].mean(axis=0)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute power.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    print 'Computing single trial power: %s %s.' %(roi, analysis)
    ltcs = np.expand_dims(ltcs,1)
    power = single_trial_power(ltcs, sfreq=Fs, frequencies=freqs, n_cycles=n_cycles, 
                                baseline=None, n_jobs=n_jobs, verbose=False)
    power = power.squeeze()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Within-trial normalization.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    power = (power.T / np.median(power, axis=-1).T).T # median, not mean

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Prepare baseline normalization (within subject, DBS).
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Compute baseline normalization.
    if analysis == 'stim':

        ## Make time mask.
        mask = (times >= baseline[0]) & (times <= baseline[1])

        ## Iteratively compute over subjects.
        blnorm = []
        for subject in df.Subject.unique():

            ## Iteratively compute over DBS conditions..
            sbl = []
            for dbs in [0,1]:

                ix, = np.where((df.Subject==subject)&(df.DBS==dbs))
                sbl.append( np.apply_over_axes(np.median, power[ix][:,:,mask], axes=[0,2]).squeeze() )

            blnorm.append(sbl)

        ## Merge.
        blnorm = np.array(blnorm)
      
        ## Save.
        np.save(os.path.join(root_dir, 'afMSIT_source_iti_%s_bl' %roi), blnorm)
        
print 'Done.'

### Compute power of source timecourses.

In [ ]:
import os
import numpy as np
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import DataFrame

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
method = 'dSPM'
h_freq = 50

## Label parameters.
rois = ['dlpfc_5-lh']

## Time-frequency parameters.
freqs = np.logspace( np.log10(2), np.log10(50), num=25)
n_cycles = 3
Fs = 1450.
decim = 14
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'

for roi in rois:

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Iteratively load and merge.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ltcs = []
    df = dict(Condition = [], Subject = [])
    
    for subject in subjects:

        ## Load NPZ.
        npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_iti_%s_%s_epochs.npz' %(subject,method,h_freq)))

        ## Get label index and extract.
        ix = npz['labels'].tolist().index(roi)
        arr = npz['ltcs'][:,ix,:]

        ## Phase-lock removal.
        for cond in [1,2,3,4]:
            ix, = np.where(npz['conds']==cond)
            arr[ix] -= arr[ix].mean(axis=0)
        
        ## Append.
        ltcs.append(arr)
        df['Condition'] += npz['conds'].tolist()
        df['Subject'] += [subject] * len(npz['conds'])

    ## Concatenate.
    ltcs = np.concatenate(ltcs, axis=0)
    times = npz['times']
    df = DataFrame(df)
    df['DBS'] = (df.Condition > 2).astype(int)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute power.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    print 'Computing single trial power: %s.' %roi
    ltcs = np.expand_dims(ltcs,1)
    power = single_trial_power(ltcs, sfreq=Fs, frequencies=freqs, n_cycles=n_cycles, 
                                baseline=None, n_jobs=n_jobs, verbose=False)
    power = power.squeeze()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Within-trial normalization.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    power = (power.T / np.median(power, axis=-1).T).T # median, not mean

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Apply baseline normalization (within subject, DBS).
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load baseline for for normalization.
    blnorm = np.load(os.path.join(root_dir, 'afMSIT_source_iti_%s_bl.npy' %roi))
    
    ## Setup index vectors.
    n_trials, n_freqs, n_times = power.shape        
    _, subj_ix = np.unique(df.Subject, return_inverse=True)
    dbs_ix = df.DBS.as_matrix()
    trial_ix = np.arange(n_trials) 

    ## Main loop.
    for i,j,k in zip(trial_ix,subj_ix,dbs_ix):

        for m in xrange(n_times):

            power[i,:,m] /= blnorm[j,k,:]

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Final preprocessing steps.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Crop times.
    mask = (times >= -1) & (times <= 1)
    power = power[:, :, mask]
    times = times[mask]

    ## Decimate
    power = power[:,:,::decim]
    times = times[::decim]
    
    ## Save.
    f = os.path.join(root_dir, 'afMSIT_source_iti_%s_spectra' %roi)
    np.savez_compressed(f, power=power, times=times, freqs=freqs, n_cycles=n_cycles, 
                        subject=df.Subject.as_matrix(), conds=df.Condition.as_matrix())

    del power

print 'Done.'

## Resting State Spectral Comparisons
### Projections: EOG

In [ ]:
import os
from mne import write_proj
from mne.preprocessing import compute_proj_eog
from mne.io import Raw

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## File params.
root_dir = '/space/sophia/2/users/EMOTE-DBS/resting/raw'
subjects = ['S2']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
raw_files = os.listdir(root_dir)
raw_files = [f for f in raw_files if f.split('_')[0] in subjects and f.endswith('raw.fif')]

for raw_file in raw_files:    
    
    ## Load files.
    raw_file = os.path.join(root_dir, raw_file)
    raw = Raw(raw_file, preload=True, verbose=False, add_eeg_ref=False)
    raw.add_eeg_average_proj()
    
    proj, _ = compute_proj_eog(raw, n_eeg = 4, average=True, filter_length='20s',
                               reject=dict(eeg=5e-4), flat=dict(eeg=5e-8),  ch_name='EOG', n_jobs=3)
    write_proj(raw_file.replace('_raw.fif', '_eog-proj.fif'), proj)

### Projections: ECG

In [ ]:
import os
from mne import read_proj, write_proj
from mne.preprocessing import compute_proj_ecg
from mne.io import Raw

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## File params.
root_dir = '/space/sophia/2/users/EMOTE-DBS/resting/raw'
subjects = ['CHDR']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
raw_files = os.listdir(root_dir)
raw_files = [f for f in raw_files if f.split('_')[0] in subjects and f.endswith('raw.fif')]

for raw_file in raw_files:    
    
    ## Load files.
    raw_file = os.path.join(root_dir, raw_file)
    raw = Raw(raw_file, preload=True, verbose=False, add_eeg_ref=False)
    raw.add_eeg_average_proj()
    
    ## Make ECG proj. Save.
    proj, _ = compute_proj_ecg(raw, n_eeg = 4, h_freq = 35., average=True, filter_length='20s',
                                reject=dict(eeg=5e-4), flat=dict(eeg=5e-8), ch_name='P9', n_jobs=3)
    write_proj(raw_file.replace('_raw.fif', '_ecg-proj.fif'), proj)

### Compute PSD

In [ ]:
import os
import numpy as np
import pylab as plt
from mne import pick_channels, read_proj, set_log_level
from mne.io import Raw
from mne.time_frequency import psd_multitaper
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define Parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
subjects = ['BRTU', 'CHDR', 'JADE', 'S2']
conds = ['resting_dbsoff_eo', 'resting_dbson_eo']

## Filtering parameters.
fmin, fmax = 0.5, 30
tmin, tmax = 10, 40
chs = ['FCZ']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/resting/raw'

psds = []
for subject in subjects:
    
    psds.append([])
    
    for cond in conds:
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Prepare data. 
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Load data.
        raw = os.path.join(root_dir, '%s_%s_raw.fif' %(subject,cond))
        raw = Raw(raw, preload=True, add_eeg_ref=False, verbose=False)
        
        ## Load/apply projection(s).
        proj = os.path.join(root_dir, '%s_%s-proj.fif' %(subject,cond))
        
        if os.path.isfile(proj): 
            proj = read_proj(proj)
            raw.add_proj(proj)
        else:
            raw.set_eeg_reference()
        
        raw.apply_proj()
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute PSD.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        picks = pick_channels(raw.ch_names, chs)
        psd, freqs = psd_multitaper(raw, fmin=fmin, fmax=fmax, tmin=10, tmax=40, picks=picks, n_jobs=3)
        psds[-1].append(psd)
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Merge and plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge.
psds = np.array(psds).squeeze()

In [ ]:
## Normalize.
psd = psds.mean(axis=0)
auc = psd.sum(axis=-1)
psd = (psd.T / auc).T

## Plot.
fig, ax = plt.subplots(1,1,figsize=(10,5))
for arr, label, color in zip(psd, ['DBSoff','DBSon'], ['#0571b0','#ca0020']):
    ax.plot(freqs, arr, linewidth=4, label=label, color=color, alpha=0.7)
ax.legend(loc=1, frameon=False, fontsize=18)

plt.tight_layout()
# plt.close('all')
plt.show()

# Section 4: Plotting (Primary Results)

## Time-Domain Results

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
labels = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
          'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
          'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']
# labels = ['FCZ']

space = 'source'
model_name = 'revised'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
img_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

for label in labels:

    for freq in [15]:

        ## Intialize figure.
        fig = plt.figure(figsize=(16,8))
        axes = []
        
        ## Initialize labels.
        if space == 'sensor': ylabel = 'Voltage (uV)'
        elif space == 'source': ylabel = 'dSPM'
        title = '%s ERP' %(label.replace('_',' ').upper())
        out_path = os.path.join(img_dir, 'timedomain', '%s_%s.png' %(model_name,title.replace(' ','_').lower()))
            
        for n, analysis in enumerate(['stim', 'resp']):
 
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Load data.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

            ## Load source data.
            npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space,analysis,label,freq)))
            data = npz['data']
            times = npz['times']
            
            ## Load cluster results.
            f = os.path.join(root_dir, 'results', '%s_%s_timedomain_results.csv' %(model_name, analysis))
            clusters = read_csv(f)
            
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Plot DBS Difference.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            
            ax = plt.subplot2grid((2,2),(n,0))
            for m, color, legend in zip([0,1],['#0571b0','#ca0020'],['DBSoff','DBSon']):
                
                ix, = np.where(info.DBS==m)
                mu = data[ix].mean(axis=0)
                se = data[ix].std(axis=0) / np.sqrt(len(ix))
                ax.plot(times, mu, linewidth=2, color=color, label=legend)
                ax.fill_between(times, mu-se, mu+se, color=color, alpha=0.2)
        
            ## Plot significant clusters.
            for ix in np.where((clusters.Label==label)&(clusters.Freq==freq)&
                               (clusters.Contrast=='DBS')&(clusters.FDR<0.05))[0]:
                y1, y2 = ax.get_ylim()
                tmin, tmax = clusters.loc[ix,'Tmin'], clusters.loc[ix,'Tmax']
                ax.fill_between(np.linspace(tmin,tmax,1e3), y1, y2, color='k', alpha=0.2) 
                ax.set_ylim(y1,y2)
                
            axes.append(ax)
                
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Plot Interference Difference.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

            ax = plt.subplot2grid((2,2),(n,1))
            for m, color, legend in zip([0,1],['#7b3294','#008837'],['Neu','Int']):
                
                ix, = np.where(info.Interference==m)
                mu = data[ix].mean(axis=0)
                se = data[ix].std(axis=0) / np.sqrt(len(ix))
                ax.plot(times, mu, linewidth=2, color=color, label=legend)
                ax.fill_between(times, mu-se, mu+se, color=color, alpha=0.2)
                
            ## Plot significant clusters.
            for ix in np.where((clusters.Label==label)&(clusters.Freq==freq)&
                               (clusters.Contrast=='Interference')&(clusters.FDR<0.05))[0]:
                y1, y2 = ax.get_ylim()
                tmin, tmax = clusters.loc[ix,'Tmin'], clusters.loc[ix,'Tmax']
                ax.fill_between(np.linspace(tmin,tmax,1e3), y1, y2, color='k', alpha=0.2)    
                ax.set_ylim(y1,y2)
            
            axes.append(ax)
            
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Add Flourishes
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        for i, ax in enumerate(axes):

            ax.legend(loc=2, fontsize=18, markerscale=2, frameon=False, borderpad=0, handletextpad=0.2)
            ax.set_xlabel('Time (s)', fontsize=18)
            if not i%2: ax.set_ylabel(ylabel, fontsize=20)
            ax.tick_params(axis='both', which='major', labelsize=12)
            if not i/2: ax.set_title('Stimulus-Locked', fontsize=24)
            else: ax.set_title('Response-Locked', fontsize=24)

            ## Time-lock specific.
            if not i/2:
                y1, y2 = ax.get_ylim()
                for x,s in zip([0, 0.4, 1.127],['IAPS','MSIT','Resp']): 
                    ax.text(x+0.02,y1+np.abs(y1*0.05),s,fontsize=16)
                    ax.vlines(x,y1,y2,linestyle='--',alpha=0.3)
                ax.set_ylim(y1,y2)

            else:
                y1, y2 = ax.get_ylim()
                ax.text(0.02,y1+np.abs(y1*0.05),'Resp',fontsize=16)
                ax.vlines(0.0,y1,y2,linestyle='--',alpha=0.3)
                ax.set_ylim(y1,y2)

        plt.suptitle(title, y=0.99, fontsize=28)
        plt.subplots_adjust(left=0.05, right=0.975, hspace=0.35, wspace=0.15)
        #plt.show()
        plt.savefig(out_path)
        plt.close()

print 'Done.'

## Power Results

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
labels = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
          'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
          'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']
# labels = ['FCZ']

space = 'source'
model_name = 'revised'
baseline = (-0.5, -0.1)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
img_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

for label in labels:

    for freq in ['theta','alpha','beta']:

        ## Intialize figure.
        fig = plt.figure(figsize=(16,8))
        axes = []
        
        ## Initialize labels.
        ylabel = 'Power (dB)'
        title = '%s %s Power' %(label.replace('_',' ').upper(), freq.capitalize())
        out_path = os.path.join(img_dir, 'frequency', '%s_%s.png' %(model_name,title.replace(' ','_').lower()))
            
        for n, analysis in enumerate(['stim', 'resp']):
 
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Load data.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

            ## Load source data.
            npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space,analysis,label,freq)))
            data = npz['data']
            times = npz['times']
            
            ## Load cluster results.
            f = os.path.join(root_dir, 'results', '%s_%s_frequency_results.csv' %(model_name, analysis))
            clusters = read_csv(f)
            
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Plot DBS Difference.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            
            ax = plt.subplot2grid((2,2),(n,0))
            for m, color, legend in zip([0,1],['#0571b0','#ca0020'],['DBSoff','DBSon']):
                
                ix, = np.where(info.DBS==m)
                mu = data[ix].mean(axis=0)
                if analysis == 'stim': mu -= mu[(times >= baseline[0])&(times <= baseline[1])].mean()
                se = data[ix].std(axis=0) / np.sqrt(len(ix))
                ax.plot(times, mu, linewidth=2, color=color, label=legend)
                ax.fill_between(times, mu-se, mu+se, color=color, alpha=0.2)
        
            ## Plot significant clusters.
            for ix in np.where((clusters.Label==label)&(clusters.Freq==freq)&
                               (clusters.Contrast=='DBS')&(clusters.FDR<0.05))[0]:
                y1, y2 = ax.get_ylim()
                tmin, tmax = clusters.loc[ix,'Tmin'], clusters.loc[ix,'Tmax']
                ax.fill_between(np.linspace(tmin,tmax,1e3), y1, y2, color='k', alpha=0.2) 
                ax.set_ylim(y1,y2)
                
            axes.append(ax)
                
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Plot Interference Difference.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

            ax = plt.subplot2grid((2,2),(n,1))
            for m, color, legend in zip([0,1],['#7b3294','#008837'],['Neu','Int']):
                
                ix, = np.where(info.Interference==m)
                mu = data[ix].mean(axis=0)
                if analysis == 'stim': mu -= mu[(times >= baseline[0])&(times <= baseline[1])].mean()
                se = data[ix].std(axis=0) / np.sqrt(len(ix))
                ax.plot(times, mu, linewidth=2, color=color, label=legend)
                ax.fill_between(times, mu-se, mu+se, color=color, alpha=0.2)
                
            ## Plot significant clusters.
            for ix in np.where((clusters.Label==label)&(clusters.Freq==freq)&
                               (clusters.Contrast=='Interference')&(clusters.FDR<0.05))[0]:
                y1, y2 = ax.get_ylim()
                tmin, tmax = clusters.loc[ix,'Tmin'], clusters.loc[ix,'Tmax']
                ax.fill_between(np.linspace(tmin,tmax,1e3), y1, y2, color='k', alpha=0.2)    
                ax.set_ylim(y1,y2)
            
            axes.append(ax)
            
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Add Flourishes
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        for i, ax in enumerate(axes):

            ax.legend(loc=2, fontsize=18, markerscale=2, frameon=False, borderpad=0, handletextpad=0.2)
            ax.set_xlabel('Time (s)', fontsize=18)
            if not i%2: ax.set_ylabel(ylabel, fontsize=20)
            ax.tick_params(axis='both', which='major', labelsize=12)
            if not i/2: ax.set_title('Stimulus-Locked', fontsize=24)
            else: ax.set_title('Response-Locked', fontsize=24)

            ## Time-lock specific.
            if not i/2:
                y1, y2 = ax.get_ylim()
                for x,s in zip([0, 0.4, 1.127],['IAPS','MSIT','Resp']): 
                    ax.text(x+0.02,y1+np.abs(y1*0.05),s,fontsize=16)
                    ax.vlines(x,y1,y2,linestyle='--',alpha=0.3)
                ax.set_ylim(y1,y2)

            else:
                y1, y2 = ax.get_ylim()
                ax.text(0.02,y1+np.abs(y1*0.05),'Resp',fontsize=16)
                ax.vlines(0.0,y1,y2,linestyle='--',alpha=0.3)
                ax.set_ylim(y1,y2)

        plt.suptitle(title, y=0.99, fontsize=28)
        plt.subplots_adjust(left=0.05, right=0.975, hspace=0.35, wspace=0.15)
        #plt.show()
        plt.savefig(out_path)
        plt.close()

print 'Done.'

# Section 5: Secondary Figures
## Intertrial Interval Spectra Comparison


In [ ]:
import os
import numpy as np
import pylab as plt
from cmap_utils import *

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and extract data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load data.
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'
npz = np.load( os.path.join(root_dir, 'afMSIT_source_iti_dlpfc_5-lh_spectra.npz') )

## Extract data.
power = npz['power']
times = npz['times']
freqs = npz['freqs']
dbs = (npz['conds'] > 2).astype(int)

## Compute contrasts.
dbs_off = np.log10( np.median( power[dbs==0], axis=0 ) ) * 10
dbs_on = np.log10( np.median( power[dbs==1], axis=0 ) ) * 10
contrast = dbs_on - dbs_off

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot spectrograms.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig, axes = plt.subplots(1,3,figsize=(15,5),sharex=True)
vmin, vmax = np.floor( np.min([dbs_off, dbs_on]) ), np.ceil( np.max([dbs_off, dbs_on]) )


## Plot DBS-off.
cmap = center_color_map(dbs_off, 'jet')
cbar = axes[0].imshow(dbs_off , aspect='auto', origin='lower', cmap=cmap, vmin=vmin, vmax=vmax)
plt.colorbar(cbar, ax=axes[0],  ticks=np.arange(vmin, vmax+0.01, 0.5))
axes[0].set_ylabel('Frequency', fontsize=18)

## Plot DBS-on.
cmap = center_color_map(dbs_on, 'jet')
cbar = axes[1].imshow(dbs_on , aspect='auto', origin='lower', cmap=cmap, vmin=vmin, vmax=vmax)
plt.colorbar(cbar, ax=axes[1], ticks=np.arange(vmin, vmax+0.01, 0.5))

## Plot contrast.
cmap = center_color_map(contrast, 'jet')
cbar = axes[2].imshow(contrast , aspect='auto', origin='lower', cmap=cmap, vmin=-1, vmax=1)
plt.colorbar(cbar, ax=axes[2], ticks=np.arange(-1,1.1,0.5))

## Add flourishes.
for ax, title in zip(axes, ['DBSoff', 'DBSon', 'On-Off']):
    
    ax.set_xticks(np.linspace(0, times.shape[0], 6))
    ax.set_xticklabels(np.linspace(times.min(), times.max(), 6).round(2))
    ax.set_xlabel('Time (s)', fontsize=16)
    
    ax.set_yticks([0,5,10,15,18,20,22,24])
    ax.set_yticklabels(freqs[[0,5,10,15,18,20,22,24]].astype(int))
    ax.set_title(title, fontsize=20)
    
plt.subplots_adjust(left=0.05, right=0.98, top=0.8)
plt.suptitle('Intertrial Power Spectra', fontsize=28)
plt.show()

## Intertrial Interval Power Spectrum Density

In [ ]:
import os
import numpy as np
import pylab as plt
from mne import EpochsArray, create_info
from mne.time_frequency import psd_multitaper

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
method = 'dSPM'
h_freq = 50
roi = 'dlpfc_5-lh'
sfreq = 1450

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'

epochs, conds = [], []
for subject in subjects:

    ## Load NPZ.
    npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_iti_%s_%s_epochs.npz' %(subject,method,h_freq)))
    epochs.append(npz['ltcs'])
    conds.append(npz['conds'])

## Concatenate.
epochs = np.concatenate(epochs, axis=0)
conds = np.concatenate(conds, axis=0)
times = npz['times']

## Make into epochs object.
n_trials, n_channels, n_times = epochs.shape

## Make info.
info = create_info([roi], sfreq, ['eeg'])

## Make events.
events = np.zeros((n_trials,3), dtype=int)
events.T[0] = np.arange(n_trials)
events.T[-1] = conds
event_id = dict( FN=1, FI=2, NN=3, NI=4 )

epochs = EpochsArray(epochs, info, events, tmin=-1.25, event_id=event_id, verbose=False)

## Compute PSD.
psd_dbsoff, freqs = psd_multitaper(epochs[['FI','FN']], fmin=0, fmax=50, n_jobs=3, verbose=False)
psd_dbson,  freqs = psd_multitaper(epochs[['NI','NN']], fmin=0, fmax=50, n_jobs=3, verbose=False)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))

for arr, label, color in zip([psd_dbsoff,psd_dbson], ['DBSoff','DBSon'], ['#0571b0','#ca0020']):
    
    
    mu = arr.squeeze().mean(axis=0)
    mask = (freqs <= 30)
    auc = mu[mask].sum()
    
    arr2 = arr / auc
    
    mu = arr2.squeeze().mean(axis=0)
    sd = arr2.squeeze().std(axis=0)
    se = sd / np.sqrt(arr2.shape[0])
    
    ax.plot(freqs, mu, linewidth=3, label=label, color=color)
    ax.fill_between(freqs, mu-se, mu+se, color=color, alpha=0.2)
    
ax.legend(loc=1)

plt.tight_layout()
plt.show()

In [ ]:
se

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot spectrograms.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig, axes = plt.subplots(1,3,figsize=(15,5),sharex=True)
vmin, vmax = np.floor( np.min([dbs_off, dbs_on]) ), np.ceil( np.max([dbs_off, dbs_on]) )


## Plot DBS-off.
cmap = center_color_map(dbs_off, 'jet')
cbar = axes[0].imshow(dbs_off , aspect='auto', origin='lower', cmap=cmap, vmin=vmin, vmax=vmax)
plt.colorbar(cbar, ax=axes[0],  ticks=np.arange(vmin, vmax+0.01, 0.5))
axes[0].set_ylabel('Frequency', fontsize=18)

## Plot DBS-on.
cmap = center_color_map(dbs_on, 'jet')
cbar = axes[1].imshow(dbs_on , aspect='auto', origin='lower', cmap=cmap, vmin=vmin, vmax=vmax)
plt.colorbar(cbar, ax=axes[1], ticks=np.arange(vmin, vmax+0.01, 0.5))

## Plot contrast.
cmap = center_color_map(contrast, 'jet')
cbar = axes[2].imshow(contrast , aspect='auto', origin='lower', cmap=cmap, vmin=-1, vmax=1)
plt.colorbar(cbar, ax=axes[2], ticks=np.arange(-1,1.1,0.5))

## Add flourishes.
for ax, title in zip(axes, ['DBSoff', 'DBSon', 'On-Off']):
    
    ax.set_xticks(np.linspace(0, times.shape[0], 6))
    ax.set_xticklabels(np.linspace(times.min(), times.max(), 6).round(2))
    ax.set_xlabel('Time (s)', fontsize=16)
    
    ax.set_yticks([0,5,10,15,18,20,22,24])
    ax.set_yticklabels(freqs[[0,5,10,15,18,20,22,24]].astype(int))
    ax.set_title(title, fontsize=20)
    
plt.subplots_adjust(left=0.05, right=0.98, top=0.8)
plt.suptitle('Intertrial Power Spectra', fontsize=28)
plt.show()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot power spectrum densities.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig, ax = plt.subplots(1,1,figsize=(8,8))

ax.plot(freqs, )

# Section 6: Movies
## Source localize evoked potentials

In [ ]:
import os
import numpy as np
import pylab as plt
from mne import BiHemiLabel, read_epochs, read_label, read_source_spaces, set_log_level
from mne.minimum_norm import apply_inverse, read_inverse_operator
from scipy.io import loadmat
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
analysis = 'resp'
parc = 'april2016'
fmax = 50

## Source localization parameters.
method = 'dSPM'
snr = 1.0  
lambda2 = 3.0 / snr ** 2 # Note increased SNR
pick_ori = 'normal'

## Labels
rois = ['dacc-lh', 'dacc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initialize.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'

## Prepare fsaverage source space.
src = read_source_spaces(os.path.join(fs_dir,'fscopy','bem','fscopy-oct-6p-src.fif'))
vertices_to = [src[n]['vertno'] for n in xrange(2)]

## Prepare BiHemiLabel.
label_dir = os.path.join(fs_dir,'fscopy','label','april2016')
label_lh = np.sum([read_label('%s/%s.label' %(label_dir,roi), subject='fsaverage') for roi in rois if roi.endswith('-lh')])
label_rh = np.sum([read_label('%s/%s.label' %(label_dir,roi), subject='fsaverage') for roi in rois if roi.endswith('-rh')])
label = BiHemiLabel(label_lh, label_rh, name='afmsit')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
for subject in subjects:

    print 'Performing source localization: %s' %subject

    ## Load in epochs.
    epochs = read_epochs(os.path.join(root_dir,'ave','%s_%s_%s_%s-epo.fif' %(subject,task,fmax,analysis)))
    times = epochs.times
    
    ## Load in secondary files.
    inv = read_inverse_operator(os.path.join(root_dir,'cov','%s_%s_%s-inv.fif' %(subject,task,fmax)))
    morph_mat = loadmat(os.path.join(root_dir, 'morph_maps', '%s-fsaverage_morph.mat' %subject))['morph_mat']

    ## Iterate over {DBS x Interference}
    for event in epochs.event_id.iterkeys():
        
        ## Compute evoked. Source localize.
        evoked = epochs[event].average()
        stc = apply_inverse(evoked, inv, method=method, lambda2=lambda2, pick_ori=pick_ori)
        
        ## Morph to common (fsaverage) space. Restrict to vertices in label.
        stc = stc.morph_precomputed('fsaverage', vertices_to=vertices_to, morph_mat=morph_mat)
        stc = stc.in_label(label)
    
        ## Save.
        f = os.path.join(root_dir,'source','stcs','%s_%s_%s_%s_%s_%s_evoked' %(subject,task,analysis,method,fmax,event))
        stc.save(f)
        
print 'Done.'

## Reassemble source localized STCs

In [ ]:
import os
import numpy as np
from mne import read_source_estimate
from mne.filter import low_pass_filter

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
analysis = 'resp'
method = 'dSPM'
h_freq = 50

## Preprocessing parameters.
fmax = 15
n_jobs = 3

## Contrast parameters.
events = ['FN','FI','NN','NI']
condict = dict(DBS = np.array([-1,-1,1,1]), Intereference = np.array([-1,1,-1,1]))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'

SPMs = []
for event in events:
        
    stcs = []
    for subject in subjects:
        
        ## Load data.
        f = os.path.join(root_dir, 'stcs', '%s_msit_%s_%s_%s_%s_evoked-lh.stc' %(subject,analysis,method,h_freq,event))
        stc = read_source_estimate(f)
        
        ## Low pass filter.
        stc._data = low_pass_filter(stc.data.astype(np.float64), stc.sfreq, fmax, filter_length='20s', n_jobs=n_jobs)
                
        ## Crop times. Downsample.
        stc.resample(500., npad='auto')
        if analysis == 'stim': stc = stc.crop(-0.5, 2.0)
        elif analysis == 'resp': stc = stc.crop(-1.0,1.0)

        ## Append.
        stcs.append(stc)
        
    ## Average. Save.
    stc = np.mean(stcs)
    SPMs.append(stc)
    stc.save(os.path.join(root_dir, 'afMSIT_source_%s_%s_%s' %(analysis, event, fmax)))
    del stcs
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute the usual contrasts.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for k,v in condict.iteritems():
    
    stc = np.dot(SPMs, v)
    stc.save(os.path.join(root_dir, 'afMSIT_source_%s_%s_%s' %(analysis, k, fmax)))
    
print 'Done.'